In [1]:
import matplotlib.pyplot as plt
from collections import Counter, OrderedDict
import numpy as np
import os
from PIL import Image
from sklearn.metrics import f1_score

In /home/holic/miniconda3/envs/tf1.11/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /home/holic/miniconda3/envs/tf1.11/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /home/holic/miniconda3/envs/tf1.11/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


In [2]:
def stitch_count(instr_path, file_path):    
    img_files = []
    if file_path:
        for f_path in file_path:
            with open(f_path, "r") as f:
                img_f = f.read().splitlines()
                img_files.extend(img_f)
    else:
        img_files = os.listdir(instr_path)

    element_counts = Counter()
    for f in img_files:
        if f.endswith('.png'):
            img_path = os.path.join(instr_path, f)
        else:
            img_path = os.path.join(instr_path, f + '.png')
        img = np.array(Image.open(img_path)).astype(np.int32)

        cnt = Counter(img.flatten())
        element_counts += cnt
    return OrderedDict(sorted(element_counts.items()))


def ordered_dict_to_list(ordered_dict, num_classes):
    """
    Convert an OrderedDict to a list, filling missing class indices with zero.

    Parameters:
    - ordered_dict (OrderedDict): The input OrderedDict with class counts.
    - num_classes (int): The total number of classes.

    Returns:
    - list: A list of length `num_classes` with counts from the OrderedDict, filling missing indices with zero.
    """
    result_list = [0] * num_classes  # Initialize a list with zeros
    for key, value in ordered_dict.items():
        if key < num_classes:
            result_list[key] = value
    return np.array(result_list).reshape(num_classes, 1)


def calculate_f1(conf_matrix):
    """
    Calculate the weighted F1 score from a given confusion matrix.

    Parameters:
    - conf_matrix (np.ndarray): 2D array representing the confusion matrix with predicted labels as columns
                                and true labels as rows.

    Returns:
    - float: The weighted F1 score.
    """
    # Extract y_true and y_pred from the confusion matrix
    y_true = []
    y_pred = []
    for true_class in range(conf_matrix.shape[0]):
        for pred_class in range(conf_matrix.shape[1]):
            count = int(round(conf_matrix[true_class, pred_class]))  # Ensure count is an integer
            y_true.extend([true_class] * count)
            y_pred.extend([pred_class] * count)

    # Calculate and return the weighted F1 score
    class_f1_weighted_scores = f1_score(y_true, y_pred, average='weighted', zero_division=0)
    
    # Calculate F1 score for each class
    class_f1_scores = f1_score(y_true, y_pred, average=None, labels=range(conf_matrix.shape[0]), zero_division=0)

    # Create a dictionary mapping each class to its F1 score
    f1_scores_dict = {class_label: f1 for class_label, f1 in enumerate(class_f1_scores)}

    return class_f1_weighted_scores, f1_scores_dict

In [3]:
#RFI_complex_a0.5
tmp=stitch_count('../dataset-mit/instruction',['../dataset-mit/val_real.txt','../dataset-mit/val_synt.txt'])
true_cnt=ordered_dict_to_list(tmp,17)
conf=np.array([
[0.937,0.041,0.001,0.003,0.006,0.000,0.006,0.000,0.000,0.000,0.000,0.000,0.001,0.001,0.002,0.001,0.000],
[0.040,0.934,0.001,0.004,0.002,0.000,0.003,0.000,0.006,0.000,0.006,0.000,0.001,0.002,0.000,0.001,0.000],
[0.196,0.064,0.567,0.173,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.220,0.095,0.092,0.570,0.003,0.002,0.009,0.000,0.003,0.000,0.004,0.000,0.000,0.001,0.000,0.000,0.000],
[0.350,0.080,0.000,0.170,0.388,0.000,0.002,0.000,0.009,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.299,0.004,0.000,0.153,0.011,0.000,0.509,0.000,0.000,0.000,0.024,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.002,0.312,0.000,0.030,0.003,0.000,0.000,0.000,0.616,0.000,0.037,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.002,0.325,0.000,0.046,0.000,0.000,0.030,0.000,0.006,0.000,0.591,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.198,0.216,0.000,0.000,0.000,0.000,0.000,0.000,0.069,0.000,0.001,0.000,0.480,0.006,0.000,0.030,0.000],
[0.291,0.159,0.000,0.018,0.000,0.000,0.000,0.000,0.000,0.000,0.025,0.000,0.015,0.420,0.061,0.003,0.008],
[0.160,0.226,0.000,0.000,0.000,0.000,0.009,0.000,0.000,0.000,0.037,0.000,0.000,0.142,0.425,0.000,0.000],
[0.147,0.230,0.000,0.000,0.000,0.000,0.000,0.000,0.017,0.000,0.017,0.000,0.104,0.000,0.006,0.478,0.000],
[0.000,0.975,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.025]])

true_conf=conf*true_cnt
class_f1_weighted_scores, f1_scores_dict = calculate_f1(true_conf)
print(class_f1_weighted_scores)
print(f1_scores_dict)

0.9017381056824617
{0: 0.9509464997234945, 1: 0.8701929920891547, 2: 0.5428216797683079, 3: 0.5359059253339422, 4: 0.4575590902333878, 5: 0.0, 6: 0.5598731165741475, 7: 0.0, 8: 0.5660621761658031, 9: 0.0, 10: 0.534815681685196, 11: 0.0, 12: 0.5209656925031766, 13: 0.42027920646583394, 14: 0.44172569220862845, 15: 0.5228658536585366, 16: 0.04952380952380952}


In [4]:
#RFINet_notran_noaug_newinst
tmp=stitch_count('../dataset-mit/instruction',['../dataset-mit/val_real.txt','../dataset-mit/val_synt.txt'])
true_cnt=ordered_dict_to_list(tmp,14)
conf=np.array([
[0.985,0.007,0.001,0.001,0.001,0.000,0.000,0.002,0.001,0.001,0.001,0.000,0.000,0.000],
[0.045,0.931,0.010,0.000,0.006,0.000,0.000,0.003,0.003,0.001,0.001,0.000,0.000,0.000],
[0.162,0.063,0.742,0.003,0.019,0.000,0.000,0.003,0.007,0.003,0.000,0.000,0.000,0.000],
[0.038,0.001,0.000,0.955,0.000,0.000,0.000,0.003,0.002,0.001,0.000,0.000,0.000,0.000],
[0.041,0.024,0.004,0.011,0.913,0.000,0.000,0.000,0.004,0.002,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.059,0.000,0.000,0.002,0.000,0.000,0.000,0.934,0.005,0.000,0.000,0.000,0.000,0.000],
[0.059,0.001,0.000,0.002,0.000,0.000,0.000,0.015,0.923,0.000,0.000,0.000,0.000,0.000],
[0.160,0.059,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.778,0.003,0.000,0.000,0.000],
[0.184,0.035,0.000,0.000,0.000,0.000,0.000,0.007,0.000,0.000,0.774,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000]])

true_conf=conf*true_cnt
class_f1_weighted_scores, f1_scores_dict = calculate_f1(true_conf)
print(class_f1_weighted_scores)
print(f1_scores_dict)

0.9726143710823633
{0: 0.984810006341969, 1: 0.9486877224199287, 2: 0.6431784107946027, 3: 0.9503443276549475, 4: 0.8985363433391218, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.7644652250146114, 9: 0.0, 10: 0.7501565435190983, 11: 0.0, 12: 0.0, 13: 0.0}


In [5]:
#RFINet_front_xferln_MIL_160k
tmp=stitch_count('../dataset/instruction-front',['../dataset/val_real.txt'])
true_cnt=ordered_dict_to_list(tmp,14)
conf=np.array([
[0.898,0.031,0.024,0.009,0.004,0.000,0.000,0.002,0.002,0.000,0.000,0.016,0.012,0.001],
[0.126,0.831,0.027,0.005,0.006,0.000,0.000,0.000,0.001,0.001,0.002,0.000,0.001,0.000],
[0.343,0.038,0.535,0.042,0.018,0.000,0.000,0.003,0.002,0.000,0.000,0.000,0.003,0.014],
[0.321,0.006,0.009,0.630,0.014,0.000,0.000,0.015,0.006,0.000,0.000,0.000,0.000,0.000],
[0.590,0.030,0.044,0.031,0.286,0.000,0.000,0.005,0.013,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.386,0.003,0.000,0.000,0.591,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.021,0.000],
[0.251,0.006,0.006,0.006,0.037,0.000,0.000,0.681,0.012,0.000,0.000,0.000,0.000,0.000],
[0.348,0.021,0.001,0.002,0.003,0.000,0.000,0.032,0.583,0.000,0.000,0.000,0.009,0.000],
[0.402,0.066,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.531,0.000,0.000,0.000,0.000],
[0.558,0.051,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.048,0.343,0.000,0.000,0.000],
[0.621,0.001,0.001,0.002,0.008,0.000,0.001,0.000,0.000,0.000,0.000,0.359,0.003,0.002],
[0.355,0.064,0.015,0.003,0.038,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.524,0.000],
[0.492,0.140,0.107,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.261]])

true_conf=conf*true_cnt
class_f1_weighted_scores, f1_scores_dict = calculate_f1(true_conf)
print(class_f1_weighted_scores)
print(f1_scores_dict)

0.8209939923534646
{0: 0.8945306753437807, 1: 0.7941496511553479, 2: 0.5607771399735066, 3: 0.6096929337772844, 4: 0.34733893557422973, 5: 0.0, 6: 0.0, 7: 0.7070630562393487, 8: 0.6555392516507703, 9: 0.656641604010025, 10: 0.48133595284872294, 11: 0.31082999870917777, 12: 0.5097475887543609, 13: 0.36042402826855124}


In [6]:
#RFINet_front_xferln_160k
tmp=stitch_count('../dataset/instruction-front',['../dataset/val_real.txt'])
true_cnt=ordered_dict_to_list(tmp,14)
conf=np.array([
[0.914,0.029,0.025,0.006,0.004,0.000,0.000,0.001,0.002,0.000,0.000,0.011,0.007,0.000],
[0.141,0.820,0.018,0.004,0.008,0.000,0.000,0.000,0.000,0.001,0.003,0.000,0.005,0.000],
[0.358,0.052,0.508,0.038,0.030,0.000,0.000,0.003,0.001,0.000,0.000,0.000,0.009,0.001],
[0.276,0.009,0.009,0.679,0.008,0.000,0.000,0.004,0.015,0.000,0.000,0.000,0.000,0.000],
[0.546,0.032,0.102,0.007,0.303,0.000,0.000,0.008,0.000,0.000,0.000,0.000,0.001,0.000],
[0.667,0.111,0.000,0.000,0.222,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.295,0.004,0.004,0.000,0.486,0.000,0.210,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.286,0.017,0.007,0.022,0.051,0.000,0.000,0.598,0.018,0.000,0.000,0.000,0.000,0.000],
[0.345,0.032,0.012,0.023,0.004,0.000,0.000,0.015,0.558,0.000,0.011,0.000,0.000,0.000],
[0.238,0.178,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.581,0.000,0.000,0.000,0.000],
[0.366,0.052,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.081,0.501,0.000,0.000,0.000],
[0.546,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.453,0.000,0.000],
[0.251,0.042,0.031,0.001,0.044,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.629,0.000],
[0.395,0.386,0.036,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.183]])

true_conf=conf*true_cnt
class_f1_weighted_scores, f1_scores_dict = calculate_f1(true_conf)
print(class_f1_weighted_scores)
print(f1_scores_dict)

0.831402695082505
{0: 0.9054924998574116, 1: 0.7816939629026595, 2: 0.5364815050209736, 3: 0.6779635142355545, 4: 0.355020261143629, 5: 0.0, 6: 0.3489583333333333, 7: 0.6906717203621815, 8: 0.6406893967778194, 9: 0.6846703733121525, 10: 0.5979899497487438, 11: 0.4298642533936652, 12: 0.6330538159431058, 13: 0.30752688172043013}


In [7]:
#RFINet_complete_MIL
tmp=stitch_count('../dataset/instruction-complete',['../dataset/val_real.txt'])
true_cnt=ordered_dict_to_list(tmp,34)
conf=np.array([
[0.895,0.043,0.004,0.003,0.000,0.001,0.000,0.010,0.008,0.003,0.003,0.009,0.000,0.000,0.000,0.001,0.016,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.000],
[0.125,0.588,0.032,0.031,0.004,0.000,0.000,0.000,0.001,0.004,0.006,0.000,0.000,0.000,0.000,0.000,0.000,0.185,0.010,0.000,0.000,0.005,0.001,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.007,0.000,0.000],
[0.499,0.195,0.163,0.108,0.000,0.000,0.000,0.000,0.000,0.004,0.004,0.000,0.000,0.000,0.012,0.000,0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.009,0.000],
[0.543,0.032,0.186,0.163,0.002,0.006,0.000,0.023,0.006,0.000,0.000,0.001,0.000,0.000,0.000,0.006,0.032,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.160,0.112,0.143,0.007,0.248,0.046,0.002,0.002,0.000,0.000,0.000,0.006,0.000,0.000,0.000,0.076,0.001,0.162,0.004,0.000,0.000,0.004,0.000,0.004,0.000,0.022,0.000,0.000,0.000,0.001,0.000,0.001,0.000,0.000],
[0.459,0.059,0.002,0.005,0.009,0.138,0.009,0.000,0.014,0.000,0.000,0.251,0.000,0.018,0.000,0.020,0.017,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.758,0.000,0.000,0.010,0.069,0.046,0.036,0.004,0.000,0.000,0.000,0.024,0.000,0.000,0.000,0.050,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.776,0.001,0.000,0.015,0.000,0.000,0.007,0.106,0.000,0.000,0.000,0.020,0.002,0.000,0.000,0.005,0.067,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.736,0.049,0.000,0.000,0.004,0.014,0.004,0.000,0.134,0.000,0.000,0.031,0.000,0.000,0.000,0.000,0.028,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.620,0.140,0.000,0.083,0.000,0.000,0.017,0.000,0.000,0.116,0.017,0.000,0.000,0.000,0.000,0.000,0.007,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.702,0.162,0.000,0.000,0.000,0.007,0.000,0.000,0.000,0.058,0.071,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.187,0.064,0.005,0.001,0.032,0.226,0.000,0.000,0.001,0.027,0.000,0.155,0.000,0.016,0.000,0.283,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.325,0.000,0.000,0.513,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.162,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.001,0.000,0.000,0.000,0.488,0.000,0.000,0.000,0.000,0.000,0.000,0.504,0.000,0.000,0.000,0.007,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.694,0.000,0.000,0.306,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.409,0.034,0.003,0.000,0.018,0.010,0.009,0.004,0.000,0.000,0.000,0.311,0.000,0.000,0.000,0.185,0.017,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.647,0.017,0.000,0.000,0.002,0.008,0.002,0.011,0.049,0.000,0.000,0.077,0.000,0.000,0.000,0.004,0.181,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.046,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.801,0.039,0.000,0.000,0.003,0.001,0.001,0.015,0.002,0.008,0.005,0.005,0.009,0.016,0.013,0.033,0.000],
[0.000,0.075,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.363,0.507,0.001,0.000,0.004,0.000,0.000,0.000,0.000,0.024,0.000,0.000,0.000,0.000,0.007,0.017,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.802,0.000,0.094,0.022,0.000,0.028,0.000,0.000,0.000,0.042,0.000,0.000,0.010,0.002,0.000,0.000,0.000],
[0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.642,0.000,0.061,0.247,0.000,0.043,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.003,0.000],
[0.000,0.143,0.000,0.000,0.112,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.606,0.053,0.000,0.003,0.012,0.000,0.000,0.000,0.034,0.034,0.000,0.000,0.001,0.000,0.001,0.000,0.000],
[0.859,0.116,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.001,0.012,0.000,0.005,0.001,0.000,0.001,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.103,0.000,0.000,0.286,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.455,0.098,0.000,0.001,0.003,0.001,0.000,0.000,0.036,0.008,0.000,0.000,0.005,0.000,0.005,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.388,0.002,0.000,0.000,0.000,0.000,0.000,0.608,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.000],
[0.000,0.129,0.000,0.000,0.049,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.504,0.091,0.000,0.000,0.002,0.000,0.014,0.000,0.129,0.000,0.000,0.000,0.010,0.000,0.072,0.000,0.000],
[0.002,0.051,0.000,0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.355,0.085,0.004,0.001,0.004,0.010,0.000,0.000,0.000,0.381,0.000,0.000,0.100,0.004,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.563,0.000,0.000,0.000,0.000,0.000,0.000,0.074,0.000,0.000,0.301,0.000,0.000,0.062,0.000,0.000,0.000],
[0.000,0.001,0.000,0.000,0.022,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.687,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.001,0.000,0.001,0.284,0.000,0.000,0.001,0.000,0.000],
[0.000,0.019,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.498,0.037,0.002,0.001,0.030,0.000,0.000,0.000,0.000,0.148,0.000,0.000,0.257,0.000,0.007,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.516,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.045,0.000,0.000,0.437,0.000,0.000,0.000],
[0.000,0.139,0.000,0.000,0.027,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.447,0.053,0.000,0.001,0.050,0.000,0.002,0.000,0.003,0.000,0.000,0.000,0.003,0.000,0.275,0.000,0.000],
[0.002,0.008,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.758,0.025,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.009,0.000,0.000,0.000,0.000,0.002,0.195,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000]])

true_conf=conf*true_cnt
class_f1_weighted_scores, f1_scores_dict = calculate_f1(true_conf)
print(class_f1_weighted_scores)
print(f1_scores_dict)

0.7157356720206056
{0: 0.8846571945177343, 1: 0.565102548166563, 2: 0.08662613981762918, 3: 0.1357202331390508, 4: 0.3124260355029586, 5: 0.16596002422774078, 6: 0.05128205128205128, 7: 0.09183128670581954, 8: 0.1205821205821206, 9: 0.12942061700526708, 10: 0.085, 11: 0.16873196401290103, 12: 0.0, 13: 0.0, 14: 0.0, 15: 0.19106773144040223, 16: 0.21469133540941568, 17: 0.8027481501935217, 18: 0.5350672356872536, 19: 0.158311345646438, 20: 0.38006230529595014, 21: 0.01612218922358931, 22: 0.0022396416573348264, 23: 0.0, 24: 0.6092970521541949, 25: 0.12121212121212122, 26: 0.3568051754167703, 27: 0.3501411859620815, 28: 0.33129411764705885, 29: 0.20456466610312762, 30: 0.33064516129032256, 31: 0.33621245326801596, 32: 0.12015379685998077, 33: 0.0}


In [8]:
#RFINet_complete
tmp=stitch_count('../dataset/instruction-complete',['../dataset/val_real.txt'])
true_cnt=ordered_dict_to_list(tmp,34)
conf=np.array([
[0.954,0.021,0.004,0.002,0.001,0.002,0.000,0.002,0.003,0.000,0.000,0.003,0.000,0.000,0.000,0.001,0.006,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.031,0.799,0.004,0.000,0.011,0.000,0.000,0.006,0.000,0.001,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.106,0.016,0.000,0.000,0.004,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.016,0.000,0.000],
[0.454,0.051,0.469,0.005,0.000,0.000,0.000,0.000,0.000,0.005,0.011,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.311,0.157,0.130,0.370,0.000,0.000,0.002,0.017,0.000,0.000,0.000,0.004,0.000,0.000,0.000,0.000,0.009,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.077,0.150,0.145,0.003,0.306,0.011,0.002,0.004,0.000,0.000,0.000,0.077,0.000,0.000,0.000,0.040,0.000,0.127,0.016,0.000,0.000,0.018,0.000,0.017,0.000,0.006,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.163,0.031,0.005,0.009,0.023,0.672,0.000,0.000,0.007,0.000,0.000,0.062,0.000,0.000,0.000,0.020,0.007,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.329,0.000,0.000,0.067,0.000,0.004,0.088,0.000,0.024,0.000,0.000,0.000,0.000,0.000,0.000,0.004,0.484,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.118,0.000,0.000,0.029,0.007,0.000,0.000,0.785,0.005,0.000,0.000,0.010,0.000,0.000,0.000,0.012,0.033,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.226,0.037,0.000,0.023,0.020,0.042,0.016,0.016,0.505,0.000,0.000,0.023,0.000,0.000,0.000,0.000,0.091,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.148,0.102,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.746,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.130,0.060,0.000,0.000,0.000,0.004,0.000,0.000,0.000,0.027,0.779,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.166,0.048,0.033,0.032,0.010,0.045,0.000,0.000,0.000,0.025,0.002,0.598,0.000,0.010,0.000,0.022,0.007,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.504,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.162,0.171,0.000,0.000,0.000,0.162,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.977,0.000,0.000,0.000,0.018,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.833,0.083,0.000,0.028,0.000,0.000,0.000,0.000,0.000,0.056,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.070,0.030,0.005,0.000,0.012,0.014,0.002,0.000,0.000,0.000,0.000,0.051,0.000,0.000,0.000,0.804,0.010,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.240,0.027,0.001,0.010,0.001,0.008,0.008,0.050,0.029,0.004,0.000,0.011,0.000,0.000,0.000,0.002,0.609,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.003,0.029,0.000,0.000,0.006,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.839,0.030,0.000,0.000,0.004,0.000,0.003,0.011,0.006,0.005,0.010,0.003,0.007,0.020,0.009,0.015,0.000],
[0.000,0.068,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.322,0.558,0.000,0.000,0.003,0.000,0.001,0.000,0.000,0.030,0.000,0.000,0.000,0.000,0.014,0.002,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.634,0.016,0.148,0.050,0.008,0.043,0.000,0.000,0.000,0.070,0.000,0.000,0.028,0.000,0.003,0.000,0.000],
[0.000,0.015,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.373,0.000,0.044,0.251,0.000,0.233,0.000,0.000,0.000,0.000,0.000,0.000,0.084,0.000,0.000,0.000,0.000],
[0.000,0.086,0.000,0.000,0.082,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.718,0.006,0.010,0.000,0.090,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.006,0.000,0.000,0.000,0.000],
[0.003,0.856,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.120,0.013,0.000,0.001,0.000,0.000,0.004,0.000,0.000,0.000,0.001,0.000,0.000,0.001,0.000,0.000,0.000,0.000],
[0.000,0.119,0.000,0.000,0.227,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.393,0.154,0.000,0.000,0.018,0.000,0.050,0.000,0.023,0.000,0.000,0.000,0.015,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.204,0.001,0.000,0.000,0.000,0.000,0.000,0.795,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000],
[0.000,0.116,0.000,0.000,0.015,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.478,0.070,0.000,0.000,0.006,0.000,0.001,0.000,0.309,0.002,0.000,0.000,0.002,0.000,0.001,0.000,0.000],
[0.039,0.150,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.171,0.078,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.394,0.000,0.000,0.168,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.383,0.000,0.000,0.000,0.000,0.000,0.000,0.033,0.000,0.000,0.467,0.000,0.000,0.117,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.008,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.775,0.001,0.000,0.000,0.001,0.000,0.003,0.000,0.010,0.000,0.001,0.200,0.000,0.000,0.000,0.000,0.000],
[0.000,0.025,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.315,0.008,0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.211,0.000,0.000,0.436,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.501,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.032,0.000,0.000,0.467,0.000,0.000,0.000],
[0.000,0.042,0.000,0.000,0.017,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.444,0.033,0.000,0.000,0.063,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.006,0.000,0.393,0.000,0.000],
[0.023,0.017,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.521,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.029,0.000,0.000,0.000,0.000,0.000,0.409,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000]])

true_conf=conf*true_cnt
class_f1_weighted_scores, f1_scores_dict = calculate_f1(true_conf)
print(class_f1_weighted_scores)
print(f1_scores_dict)

0.8079517598466229
{0: 0.953165549457224, 1: 0.7471040647026588, 2: 0.2964641885766092, 3: 0.41690140845070417, 4: 0.3477487097348283, 5: 0.7081897694142594, 6: 0.125, 7: 0.6331006979062811, 8: 0.517200474495848, 9: 0.7696590118302019, 10: 0.8320839580209894, 11: 0.647972972972973, 12: 0.2857142857142857, 13: 0.0, 14: 0.0, 15: 0.8044660750071572, 16: 0.6482667980942993, 17: 0.8432420762129156, 18: 0.5972965246865684, 19: 0.24516129032258066, 20: 0.3797856049004594, 21: 0.10670731707317074, 22: 0.007083825265643448, 23: 0.04395604395604395, 24: 0.7669468398646436, 25: 0.1878453038674033, 26: 0.3850544888427608, 27: 0.41607418856259654, 28: 0.27464008859357697, 29: 0.3259318001586043, 30: 0.31042128603104213, 31: 0.4570657507360157, 32: 0.3625892651462797, 33: 0.0}


In [9]:
#xfer_complete_frompred_MIL
tmp=stitch_count('../dataset/instruction-complete',['../dataset/val_real.txt'])
true_cnt=ordered_dict_to_list(tmp,34)
conf=np.array([
[0.775,0.069,0.000,0.006,0.005,0.009,0.000,0.006,0.003,0.004,0.006,0.009,0.000,0.000,0.000,0.008,0.018,0.077,0.001,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.391,0.447,0.003,0.009,0.005,0.008,0.000,0.007,0.001,0.012,0.005,0.004,0.000,0.001,0.000,0.002,0.007,0.074,0.016,0.000,0.000,0.000,0.001,0.000,0.001,0.000,0.001,0.000,0.000,0.000,0.000,0.002,0.000,0.000],
[0.371,0.396,0.038,0.007,0.108,0.005,0.000,0.001,0.000,0.002,0.000,0.039,0.000,0.006,0.000,0.000,0.006,0.014,0.003,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.298,0.099,0.001,0.129,0.124,0.025,0.002,0.004,0.001,0.002,0.049,0.017,0.000,0.000,0.000,0.006,0.014,0.163,0.006,0.000,0.000,0.000,0.002,0.000,0.043,0.000,0.000,0.000,0.000,0.000,0.016,0.000,0.000,0.000],
[0.334,0.089,0.002,0.055,0.204,0.021,0.004,0.001,0.001,0.005,0.001,0.028,0.000,0.001,0.000,0.029,0.013,0.154,0.014,0.000,0.000,0.000,0.004,0.001,0.036,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000],
[0.707,0.055,0.003,0.005,0.047,0.043,0.000,0.001,0.000,0.001,0.001,0.013,0.000,0.000,0.000,0.061,0.029,0.032,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.316,0.018,0.000,0.038,0.339,0.011,0.000,0.005,0.000,0.000,0.000,0.213,0.000,0.000,0.000,0.000,0.010,0.035,0.000,0.000,0.000,0.000,0.004,0.000,0.011,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.743,0.122,0.000,0.002,0.003,0.004,0.000,0.018,0.006,0.000,0.000,0.026,0.000,0.002,0.000,0.004,0.003,0.059,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.005,0.000,0.000],
[0.607,0.118,0.002,0.001,0.032,0.008,0.000,0.009,0.007,0.001,0.000,0.101,0.000,0.000,0.000,0.002,0.004,0.067,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.039,0.000,0.000],
[0.873,0.063,0.000,0.000,0.001,0.003,0.000,0.000,0.001,0.004,0.005,0.008,0.000,0.001,0.000,0.001,0.003,0.038,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.521,0.407,0.000,0.000,0.000,0.004,0.000,0.000,0.000,0.021,0.004,0.009,0.000,0.000,0.000,0.000,0.001,0.032,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.470,0.054,0.000,0.004,0.013,0.078,0.001,0.031,0.004,0.002,0.006,0.042,0.000,0.000,0.000,0.127,0.115,0.047,0.000,0.000,0.000,0.000,0.006,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.451,0.003,0.000,0.055,0.315,0.000,0.000,0.000,0.000,0.000,0.000,0.082,0.000,0.000,0.000,0.000,0.034,0.053,0.000,0.000,0.000,0.000,0.000,0.000,0.007,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.371,0.293,0.030,0.001,0.000,0.043,0.000,0.059,0.041,0.044,0.001,0.006,0.000,0.007,0.000,0.020,0.035,0.039,0.000,0.000,0.000,0.000,0.010,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.463,0.134,0.000,0.057,0.007,0.000,0.000,0.000,0.000,0.002,0.000,0.026,0.000,0.000,0.000,0.060,0.000,0.251,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.734,0.035,0.000,0.002,0.019,0.046,0.000,0.002,0.002,0.000,0.000,0.020,0.000,0.000,0.000,0.083,0.020,0.029,0.000,0.000,0.000,0.000,0.002,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.384,0.028,0.000,0.003,0.044,0.016,0.000,0.001,0.001,0.002,0.000,0.341,0.000,0.008,0.000,0.018,0.111,0.032,0.001,0.000,0.000,0.000,0.003,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.005,0.000,0.000],
[0.594,0.059,0.000,0.008,0.009,0.001,0.000,0.000,0.000,0.001,0.002,0.001,0.000,0.000,0.000,0.001,0.003,0.232,0.041,0.000,0.000,0.000,0.000,0.000,0.020,0.000,0.001,0.005,0.001,0.001,0.000,0.019,0.001,0.000],
[0.162,0.085,0.002,0.010,0.011,0.001,0.000,0.000,0.000,0.001,0.001,0.000,0.000,0.000,0.000,0.001,0.000,0.285,0.424,0.000,0.000,0.000,0.000,0.000,0.012,0.000,0.001,0.000,0.000,0.000,0.000,0.003,0.000,0.000],
[0.266,0.037,0.001,0.077,0.025,0.001,0.000,0.000,0.000,0.000,0.000,0.023,0.000,0.000,0.000,0.013,0.004,0.171,0.148,0.000,0.000,0.000,0.001,0.000,0.233,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.086,0.168,0.000,0.091,0.164,0.000,0.001,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.001,0.221,0.135,0.000,0.000,0.000,0.001,0.000,0.130,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.272,0.060,0.003,0.086,0.090,0.001,0.000,0.000,0.000,0.000,0.000,0.006,0.000,0.000,0.000,0.004,0.001,0.329,0.040,0.000,0.000,0.000,0.000,0.000,0.098,0.000,0.000,0.000,0.000,0.000,0.000,0.009,0.000,0.000],
[0.266,0.260,0.000,0.027,0.054,0.003,0.000,0.000,0.000,0.001,0.000,0.131,0.000,0.088,0.000,0.000,0.015,0.050,0.049,0.000,0.000,0.000,0.006,0.000,0.046,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.000],
[0.250,0.132,0.000,0.090,0.066,0.032,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.009,0.002,0.298,0.028,0.000,0.000,0.000,0.001,0.026,0.056,0.001,0.000,0.000,0.000,0.000,0.000,0.006,0.000,0.000],
[0.039,0.009,0.000,0.008,0.011,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.367,0.015,0.000,0.000,0.000,0.000,0.000,0.540,0.000,0.000,0.000,0.000,0.000,0.000,0.008,0.000,0.000],
[0.209,0.028,0.003,0.204,0.157,0.015,0.000,0.000,0.000,0.001,0.005,0.014,0.000,0.000,0.000,0.040,0.004,0.153,0.017,0.000,0.000,0.000,0.000,0.038,0.104,0.001,0.000,0.000,0.000,0.000,0.000,0.009,0.000,0.000],
[0.086,0.255,0.000,0.001,0.009,0.002,0.000,0.000,0.000,0.001,0.002,0.001,0.000,0.000,0.000,0.000,0.002,0.338,0.129,0.000,0.000,0.000,0.000,0.000,0.009,0.000,0.034,0.000,0.002,0.019,0.000,0.090,0.020,0.000],
[0.090,0.014,0.000,0.000,0.001,0.001,0.000,0.000,0.001,0.004,0.021,0.003,0.000,0.000,0.000,0.000,0.002,0.582,0.000,0.000,0.000,0.000,0.000,0.001,0.003,0.000,0.001,0.139,0.023,0.000,0.009,0.106,0.000,0.000],
[0.218,0.009,0.000,0.000,0.001,0.007,0.000,0.001,0.000,0.009,0.023,0.006,0.000,0.000,0.000,0.000,0.008,0.575,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.053,0.017,0.001,0.003,0.068,0.000,0.000],
[0.061,0.239,0.000,0.000,0.008,0.002,0.000,0.000,0.000,0.001,0.003,0.001,0.000,0.001,0.000,0.000,0.002,0.381,0.106,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.031,0.001,0.001,0.058,0.000,0.076,0.026,0.000],
[0.247,0.346,0.001,0.001,0.000,0.002,0.000,0.000,0.002,0.002,0.005,0.003,0.000,0.000,0.000,0.000,0.001,0.307,0.003,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.001,0.024,0.008,0.000,0.001,0.045,0.000,0.000],
[0.029,0.033,0.000,0.006,0.003,0.001,0.000,0.001,0.000,0.001,0.001,0.000,0.000,0.000,0.000,0.000,0.001,0.352,0.002,0.000,0.000,0.000,0.000,0.000,0.006,0.000,0.007,0.001,0.000,0.001,0.000,0.553,0.001,0.000],
[0.189,0.259,0.000,0.002,0.005,0.003,0.000,0.000,0.000,0.001,0.002,0.002,0.000,0.000,0.000,0.000,0.002,0.325,0.112,0.000,0.000,0.000,0.000,0.000,0.007,0.000,0.011,0.000,0.001,0.044,0.000,0.018,0.014,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000]])

true_conf=conf*true_cnt
class_f1_weighted_scores, f1_scores_dict = calculate_f1(true_conf)
print(class_f1_weighted_scores)
print(f1_scores_dict)

0.3938373067223805
{0: 0.5662748391838252, 1: 0.41395444592090735, 2: 0.06375442739079101, 3: 0.07294317217981341, 4: 0.1980529358077274, 5: 0.04534542544678582, 6: 0.0, 7: 0.017942583732057416, 8: 0.010008340283569641, 9: 0.0037359900373599006, 10: 0.0033898305084745766, 11: 0.04530961570733344, 12: 0.0, 13: 0.0, 14: 0.0, 15: 0.08278618327148159, 16: 0.11426757657514681, 17: 0.3236692944165744, 18: 0.45376229943249086, 19: 0.0, 20: 0.0, 21: 0.0, 22: 0.01034928848641656, 23: 0.04519774011299434, 24: 0.4826051779935275, 25: 0.0, 26: 0.05871559633027523, 27: 0.1752080595707403, 28: 0.029640084685956247, 29: 0.08935824532900082, 30: 0.002602472348731295, 31: 0.515482158655264, 32: 0.02547169811320755, 33: 0.0}


In [10]:
#xfer_complete_frompred
tmp=stitch_count('../dataset/instruction-complete',['../dataset/val_real.txt'])
true_cnt=ordered_dict_to_list(tmp,34)
conf=np.array([
[0.946,0.007,0.000,0.001,0.001,0.001,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.001,0.007,0.029,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.091,0.843,0.001,0.003,0.002,0.002,0.000,0.000,0.000,0.001,0.001,0.001,0.000,0.000,0.000,0.001,0.001,0.035,0.014,0.000,0.000,0.000,0.001,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000],
[0.113,0.175,0.318,0.014,0.072,0.001,0.000,0.000,0.000,0.000,0.000,0.007,0.000,0.000,0.000,0.007,0.001,0.108,0.173,0.000,0.000,0.000,0.008,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.102,0.020,0.006,0.537,0.205,0.007,0.001,0.002,0.000,0.000,0.001,0.005,0.000,0.000,0.000,0.014,0.004,0.037,0.010,0.000,0.000,0.000,0.001,0.000,0.048,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.107,0.044,0.002,0.101,0.569,0.005,0.003,0.000,0.000,0.001,0.006,0.017,0.000,0.000,0.000,0.007,0.004,0.059,0.017,0.000,0.000,0.000,0.001,0.000,0.055,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000],
[0.069,0.007,0.001,0.003,0.003,0.725,0.000,0.042,0.001,0.000,0.011,0.015,0.000,0.000,0.000,0.076,0.006,0.027,0.001,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.009,0.000,0.000],
[0.095,0.011,0.000,0.031,0.381,0.001,0.334,0.013,0.001,0.000,0.000,0.008,0.000,0.000,0.000,0.016,0.011,0.016,0.001,0.000,0.000,0.000,0.053,0.000,0.027,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.067,0.010,0.000,0.006,0.008,0.027,0.000,0.702,0.024,0.002,0.002,0.003,0.000,0.000,0.000,0.053,0.011,0.057,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.002,0.000,0.001,0.000,0.001,0.023,0.000,0.000],
[0.068,0.010,0.000,0.010,0.002,0.047,0.000,0.107,0.411,0.073,0.036,0.005,0.000,0.000,0.000,0.068,0.014,0.072,0.001,0.000,0.000,0.000,0.012,0.000,0.000,0.001,0.004,0.001,0.001,0.000,0.001,0.055,0.000,0.000],
[0.021,0.011,0.000,0.000,0.001,0.015,0.000,0.003,0.030,0.837,0.033,0.001,0.000,0.000,0.000,0.004,0.002,0.012,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.024,0.003,0.000,0.001,0.000,0.000,0.000],
[0.022,0.020,0.000,0.000,0.001,0.019,0.000,0.001,0.016,0.026,0.858,0.001,0.000,0.000,0.000,0.000,0.002,0.014,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.006,0.014,0.000,0.000,0.000,0.000,0.000],
[0.196,0.020,0.003,0.001,0.017,0.015,0.004,0.002,0.000,0.000,0.000,0.647,0.000,0.002,0.000,0.016,0.020,0.032,0.023,0.000,0.000,0.000,0.001,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.229,0.066,0.000,0.029,0.269,0.000,0.030,0.071,0.000,0.000,0.000,0.064,0.208,0.000,0.000,0.000,0.023,0.000,0.000,0.000,0.000,0.000,0.011,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.208,0.296,0.000,0.000,0.021,0.013,0.000,0.000,0.000,0.000,0.000,0.141,0.000,0.310,0.000,0.000,0.000,0.001,0.010,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.311,0.120,0.022,0.369,0.031,0.000,0.018,0.002,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.003,0.000,0.101,0.015,0.000,0.000,0.000,0.000,0.000,0.007,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.066,0.008,0.000,0.008,0.014,0.089,0.000,0.012,0.016,0.004,0.000,0.014,0.000,0.000,0.000,0.732,0.004,0.019,0.001,0.000,0.000,0.000,0.002,0.001,0.002,0.001,0.001,0.000,0.000,0.000,0.000,0.005,0.000,0.000],
[0.314,0.008,0.000,0.001,0.004,0.005,0.003,0.006,0.002,0.001,0.000,0.036,0.000,0.000,0.000,0.005,0.573,0.039,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.650,0.045,0.000,0.003,0.002,0.001,0.000,0.000,0.000,0.001,0.001,0.001,0.000,0.000,0.000,0.000,0.001,0.248,0.025,0.000,0.000,0.000,0.000,0.000,0.010,0.000,0.000,0.004,0.001,0.000,0.000,0.006,0.000,0.000],
[0.139,0.087,0.003,0.004,0.005,0.001,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.001,0.188,0.562,0.000,0.000,0.000,0.001,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.003,0.001,0.000],
[0.184,0.115,0.009,0.024,0.140,0.000,0.000,0.008,0.000,0.000,0.000,0.041,0.000,0.000,0.000,0.009,0.000,0.257,0.072,0.000,0.000,0.000,0.001,0.000,0.140,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.045,0.194,0.000,0.084,0.365,0.001,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.002,0.000,0.166,0.066,0.000,0.000,0.000,0.001,0.000,0.075,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.175,0.085,0.002,0.229,0.190,0.001,0.001,0.000,0.000,0.000,0.000,0.008,0.000,0.000,0.000,0.008,0.001,0.212,0.013,0.000,0.000,0.000,0.000,0.000,0.062,0.000,0.002,0.000,0.001,0.000,0.000,0.007,0.001,0.000],
[0.042,0.518,0.000,0.061,0.008,0.007,0.000,0.014,0.003,0.001,0.000,0.001,0.000,0.003,0.000,0.014,0.058,0.040,0.034,0.000,0.000,0.000,0.179,0.000,0.017,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.143,0.066,0.000,0.300,0.056,0.041,0.000,0.005,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.036,0.002,0.188,0.019,0.000,0.000,0.000,0.000,0.070,0.063,0.009,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.043,0.007,0.001,0.007,0.040,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.163,0.040,0.000,0.000,0.000,0.000,0.000,0.686,0.000,0.002,0.000,0.000,0.000,0.000,0.008,0.000,0.000],
[0.250,0.051,0.001,0.202,0.027,0.076,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.035,0.002,0.219,0.027,0.000,0.000,0.002,0.000,0.000,0.040,0.066,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000],
[0.065,0.208,0.000,0.000,0.000,0.007,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.006,0.298,0.128,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.042,0.000,0.002,0.056,0.000,0.142,0.044,0.000],
[0.082,0.002,0.000,0.000,0.000,0.005,0.000,0.000,0.002,0.021,0.023,0.001,0.000,0.000,0.000,0.000,0.006,0.491,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.255,0.047,0.000,0.001,0.063,0.001,0.000],
[0.183,0.002,0.000,0.000,0.000,0.008,0.000,0.000,0.005,0.027,0.030,0.004,0.000,0.000,0.000,0.000,0.006,0.514,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.117,0.058,0.000,0.000,0.042,0.000,0.000],
[0.070,0.287,0.000,0.000,0.000,0.004,0.000,0.000,0.000,0.001,0.001,0.001,0.000,0.000,0.000,0.000,0.001,0.277,0.069,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.034,0.004,0.001,0.081,0.000,0.085,0.083,0.000],
[0.207,0.269,0.000,0.001,0.000,0.006,0.000,0.000,0.004,0.021,0.017,0.003,0.000,0.000,0.000,0.000,0.007,0.271,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.116,0.032,0.000,0.001,0.043,0.000,0.000],
[0.031,0.028,0.000,0.003,0.003,0.004,0.000,0.000,0.000,0.002,0.005,0.000,0.000,0.000,0.000,0.000,0.001,0.141,0.002,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.008,0.005,0.007,0.003,0.000,0.749,0.005,0.000],
[0.105,0.325,0.000,0.001,0.001,0.006,0.000,0.000,0.000,0.000,0.001,0.001,0.000,0.000,0.000,0.000,0.002,0.232,0.217,0.000,0.000,0.000,0.003,0.000,0.003,0.000,0.010,0.000,0.000,0.038,0.000,0.014,0.041,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000]])

true_conf=conf*true_cnt
class_f1_weighted_scores, f1_scores_dict = calculate_f1(true_conf)
print(class_f1_weighted_scores)
print(f1_scores_dict)

0.5272412783480616
{0: 0.6738708521803487, 1: 0.7559075094230212, 2: 0.43401759530791784, 3: 0.342348075087496, 4: 0.5470419990967936, 5: 0.6968085106382977, 6: 0.4274193548387097, 7: 0.6956521739130435, 8: 0.5362537764350452, 9: 0.7307012374779023, 10: 0.7285714285714285, 11: 0.6914361879373543, 12: 0.4, 13: 0.391304347826087, 14: 0.0, 15: 0.7187412194436639, 16: 0.6101527846229671, 17: 0.3653085511064641, 18: 0.5919318547090968, 19: 0.0, 20: 0.0, 21: 0.0, 22: 0.27382256297918944, 23: 0.1271676300578035, 24: 0.6658611599297012, 25: 0.12106537530266342, 26: 0.07632850241545894, 27: 0.2816577129700691, 28: 0.09090909090909093, 29: 0.12572373862696443, 30: 0.0026560424966799467, 31: 0.73095924248662, 32: 0.07165532879818594, 33: 0.0}


In [11]:
#xfer_complete_fromtrue_MIL
tmp=stitch_count('../dataset/instruction-complete',['../dataset/val_real.txt'])
true_cnt=ordered_dict_to_list(tmp,34)
conf=np.array([
[0.771,0.069,0.001,0.006,0.005,0.008,0.000,0.007,0.003,0.006,0.005,0.010,0.000,0.000,0.000,0.010,0.017,0.079,0.002,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.423,0.429,0.004,0.018,0.009,0.007,0.000,0.008,0.003,0.005,0.014,0.004,0.000,0.001,0.000,0.005,0.006,0.048,0.014,0.000,0.000,0.000,0.001,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000],
[0.301,0.368,0.040,0.032,0.112,0.000,0.000,0.003,0.000,0.000,0.007,0.091,0.000,0.009,0.000,0.003,0.004,0.019,0.011,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.318,0.113,0.000,0.130,0.107,0.025,0.003,0.007,0.006,0.016,0.041,0.007,0.000,0.000,0.000,0.011,0.002,0.184,0.004,0.000,0.000,0.000,0.004,0.000,0.020,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000],
[0.294,0.123,0.012,0.032,0.228,0.038,0.003,0.004,0.000,0.000,0.001,0.024,0.000,0.000,0.000,0.025,0.000,0.119,0.023,0.000,0.000,0.001,0.003,0.000,0.040,0.024,0.001,0.000,0.000,0.000,0.000,0.005,0.000,0.000],
[0.720,0.058,0.000,0.003,0.009,0.070,0.000,0.000,0.000,0.000,0.000,0.014,0.000,0.000,0.000,0.058,0.022,0.039,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.005,0.000,0.000],
[0.146,0.029,0.000,0.025,0.377,0.005,0.011,0.000,0.000,0.000,0.000,0.359,0.000,0.000,0.000,0.000,0.001,0.016,0.000,0.000,0.000,0.011,0.005,0.000,0.013,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.669,0.126,0.000,0.001,0.023,0.000,0.000,0.007,0.015,0.000,0.000,0.095,0.000,0.000,0.000,0.000,0.004,0.030,0.000,0.000,0.000,0.001,0.007,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.020,0.000,0.000],
[0.743,0.122,0.000,0.002,0.002,0.001,0.000,0.000,0.020,0.000,0.000,0.012,0.000,0.000,0.000,0.000,0.014,0.057,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.023,0.000,0.000],
[0.469,0.416,0.000,0.001,0.000,0.001,0.000,0.000,0.000,0.000,0.017,0.010,0.000,0.000,0.000,0.005,0.000,0.077,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.812,0.093,0.000,0.001,0.003,0.000,0.000,0.000,0.001,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.086,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.465,0.053,0.001,0.005,0.018,0.114,0.002,0.038,0.011,0.010,0.002,0.051,0.000,0.000,0.000,0.071,0.130,0.017,0.001,0.000,0.000,0.000,0.007,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.533,0.029,0.000,0.004,0.380,0.000,0.000,0.000,0.000,0.000,0.000,0.006,0.000,0.000,0.000,0.000,0.000,0.047,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.393,0.330,0.000,0.008,0.000,0.047,0.000,0.036,0.035,0.056,0.005,0.003,0.000,0.005,0.000,0.017,0.025,0.002,0.000,0.000,0.000,0.000,0.036,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.256,0.299,0.000,0.210,0.001,0.000,0.000,0.065,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.166,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.707,0.058,0.009,0.011,0.039,0.078,0.000,0.000,0.000,0.000,0.003,0.005,0.000,0.000,0.000,0.056,0.016,0.018,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.368,0.033,0.000,0.001,0.039,0.017,0.001,0.001,0.000,0.001,0.002,0.367,0.000,0.006,0.000,0.015,0.128,0.012,0.000,0.000,0.000,0.000,0.002,0.004,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000],
[0.564,0.050,0.000,0.006,0.007,0.000,0.000,0.001,0.001,0.001,0.001,0.000,0.000,0.000,0.000,0.001,0.002,0.252,0.042,0.000,0.000,0.001,0.000,0.001,0.022,0.000,0.001,0.015,0.002,0.002,0.000,0.021,0.004,0.000],
[0.148,0.064,0.006,0.003,0.010,0.001,0.000,0.001,0.001,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.282,0.450,0.000,0.000,0.000,0.002,0.001,0.024,0.001,0.000,0.000,0.000,0.000,0.000,0.003,0.001,0.000],
[0.397,0.074,0.000,0.110,0.035,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.174,0.149,0.000,0.000,0.000,0.000,0.000,0.061,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.083,0.122,0.000,0.027,0.174,0.000,0.000,0.000,0.002,0.000,0.000,0.011,0.000,0.000,0.000,0.001,0.001,0.292,0.099,0.003,0.004,0.001,0.000,0.000,0.147,0.000,0.000,0.000,0.000,0.000,0.000,0.034,0.000,0.000],
[0.233,0.046,0.003,0.033,0.082,0.000,0.000,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.011,0.002,0.385,0.022,0.000,0.000,0.001,0.001,0.000,0.141,0.000,0.001,0.000,0.000,0.000,0.000,0.033,0.000,0.000],
[0.332,0.215,0.000,0.023,0.018,0.014,0.000,0.000,0.000,0.000,0.005,0.118,0.000,0.073,0.000,0.000,0.024,0.089,0.028,0.000,0.000,0.000,0.049,0.003,0.008,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.077,0.006,0.000,0.004,0.752,0.032,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.015,0.037,0.000,0.000,0.000,0.030,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.048,0.000,0.000],
[0.018,0.005,0.000,0.001,0.017,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.367,0.017,0.000,0.000,0.000,0.000,0.000,0.571,0.000,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000],
[0.139,0.090,0.000,0.030,0.021,0.089,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.002,0.017,0.580,0.024,0.000,0.000,0.000,0.001,0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000],
[0.055,0.271,0.002,0.000,0.001,0.004,0.000,0.000,0.002,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.004,0.314,0.131,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.024,0.003,0.004,0.009,0.000,0.160,0.012,0.000],
[0.043,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.686,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.063,0.185,0.000,0.000,0.023,0.000,0.000],
[0.067,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.675,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.107,0.128,0.000,0.000,0.022,0.000,0.000],
[0.051,0.312,0.000,0.000,0.000,0.003,0.000,0.000,0.001,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.003,0.336,0.085,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.023,0.000,0.096,0.086,0.000],
[0.230,0.382,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.305,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.031,0.045,0.000,0.000,0.007,0.000,0.000],
[0.005,0.001,0.000,0.001,0.002,0.002,0.000,0.000,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.336,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.001,0.005,0.000,0.002,0.000,0.000,0.632,0.005,0.000],
[0.069,0.215,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.009,0.305,0.173,0.000,0.000,0.000,0.000,0.000,0.010,0.000,0.007,0.004,0.007,0.019,0.000,0.085,0.096,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000]])

true_conf=conf*true_cnt
class_f1_weighted_scores, f1_scores_dict = calculate_f1(true_conf)
print(class_f1_weighted_scores)
print(f1_scores_dict)

0.40799612378873057
{0: 0.572142159958661, 1: 0.4091355883203238, 2: 0.05283018867924528, 3: 0.08238276299112801, 4: 0.2206060606060606, 5: 0.07277486910994763, 6: 0.02173913043478261, 7: 0.006237006237006237, 8: 0.023776223776223775, 9: 0.0, 10: 0.0011614401858304297, 11: 0.05484949832775919, 12: 0.0, 13: 0.0, 14: 0.0, 15: 0.05617655488678704, 16: 0.1355987583728149, 17: 0.3484251714296783, 18: 0.47074786495330834, 19: 0.0, 20: 0.008097165991902834, 21: 0.002399520095980804, 22: 0.07888631090487239, 23: 0.0, 24: 0.4914775494302133, 25: 0.0, 26: 0.04316546762589928, 27: 0.0546218487394958, 28: 0.16084977238239756, 29: 0.03571428571428571, 30: 0.0, 31: 0.5535334872979214, 32: 0.13785394932935915, 33: 0.0}


In [12]:
#xfer_complete_fromtrue
tmp=stitch_count('../dataset/instruction-complete',['../dataset/val_real.txt'])
true_cnt=ordered_dict_to_list(tmp,34)
conf=np.array([
[0.960,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.006,0.030,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.057,0.913,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.005,0.000,0.001,0.000,0.000,0.000,0.014,0.008,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.015,0.043,0.500,0.000,0.023,0.000,0.001,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.005,0.403,0.000,0.000,0.000,0.008,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.001,0.000,0.002,0.705,0.155,0.000,0.006,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.078,0.000,0.004,0.001,0.000,0.000,0.001,0.011,0.000,0.032,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.013,0.015,0.001,0.091,0.725,0.000,0.009,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.029,0.000,0.041,0.002,0.000,0.000,0.000,0.000,0.000,0.073,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.001,0.000,0.852,0.000,0.034,0.002,0.000,0.031,0.000,0.000,0.000,0.000,0.062,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.008,0.001,0.000,0.000,0.000,0.000,0.007,0.000,0.000],
[0.000,0.000,0.000,0.077,0.408,0.000,0.472,0.005,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.036,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.004,0.001,0.036,0.000,0.861,0.045,0.013,0.010,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.001,0.001,0.004,0.000,0.010,0.001,0.000,0.003,0.000,0.001,0.007,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.086,0.000,0.278,0.338,0.132,0.023,0.000,0.000,0.000,0.000,0.078,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.001,0.002,0.012,0.003,0.001,0.042,0.002,0.000],
[0.000,0.000,0.000,0.000,0.001,0.010,0.000,0.003,0.041,0.865,0.021,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.003,0.024,0.028,0.002,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.020,0.000,0.001,0.004,0.027,0.894,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.027,0.023,0.000,0.000,0.000,0.000,0.000],
[0.158,0.002,0.000,0.000,0.007,0.000,0.000,0.000,0.000,0.000,0.000,0.758,0.000,0.000,0.000,0.000,0.016,0.033,0.023,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.041,0.276,0.000,0.128,0.304,0.000,0.000,0.000,0.000,0.168,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.080,0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.016,0.384,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.064,0.001,0.522,0.000,0.000,0.005,0.008,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.585,0.000,0.000,0.000,0.059,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.353,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.001,0.000,0.056,0.000,0.012,0.022,0.006,0.001,0.000,0.000,0.000,0.000,0.889,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.007,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000],
[0.261,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.046,0.000,0.000,0.000,0.000,0.667,0.019,0.000,0.000,0.000,0.000,0.007,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.613,0.086,0.000,0.002,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.003,0.271,0.013,0.000,0.000,0.000,0.000,0.000,0.010,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.116,0.003,0.007,0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.048,0.819,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.230,0.311,0.000,0.000,0.010,0.000,0.000,0.000,0.000,0.000,0.000,0.045,0.000,0.001,0.000,0.000,0.000,0.160,0.146,0.096,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.013,0.237,0.000,0.214,0.412,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.073,0.000,0.000,0.000,0.000,0.000,0.000,0.051,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.142,0.000,0.002,0.354,0.198,0.000,0.000,0.036,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.026,0.000,0.130,0.000,0.000,0.000,0.002,0.008,0.000,0.100,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.554,0.000,0.046,0.000,0.000,0.000,0.056,0.003,0.000,0.000,0.001,0.000,0.000,0.000,0.022,0.001,0.011,0.051,0.000,0.000,0.000,0.244,0.000,0.000,0.001,0.006,0.000,0.001,0.000,0.000,0.000,0.002,0.000],
[0.000,0.088,0.000,0.066,0.020,0.000,0.000,0.032,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.044,0.000,0.008,0.039,0.000,0.000,0.000,0.004,0.675,0.001,0.022,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.004,0.008,0.015,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.005,0.005,0.000,0.000,0.000,0.002,0.000,0.962,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.030,0.002,0.000,0.000,0.006,0.113,0.000,0.010,0.019,0.000,0.000,0.002,0.000,0.000,0.000,0.050,0.000,0.003,0.147,0.000,0.000,0.000,0.005,0.008,0.010,0.597,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.198,0.000,0.004,0.002,0.046,0.001,0.000,0.018,0.000,0.016,0.000,0.000,0.000,0.000,0.000,0.000,0.003,0.298,0.000,0.000,0.000,0.000,0.000,0.000,0.007,0.051,0.001,0.008,0.079,0.000,0.214,0.053,0.000],
[0.000,0.000,0.000,0.000,0.000,0.007,0.000,0.003,0.024,0.031,0.045,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.534,0.330,0.001,0.011,0.012,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.004,0.000,0.001,0.002,0.024,0.027,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.299,0.578,0.002,0.004,0.057,0.000,0.000],
[0.000,0.182,0.000,0.004,0.006,0.054,0.002,0.000,0.016,0.007,0.014,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.312,0.000,0.000,0.000,0.000,0.000,0.000,0.011,0.028,0.006,0.009,0.149,0.000,0.131,0.069,0.000],
[0.370,0.000,0.000,0.000,0.000,0.004,0.000,0.000,0.006,0.022,0.025,0.000,0.000,0.000,0.000,0.000,0.000,0.166,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.220,0.173,0.001,0.004,0.007,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.003,0.002,0.004,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.004,0.016,0.001,0.011,0.014,0.000,0.932,0.007,0.000],
[0.000,0.112,0.000,0.000,0.000,0.001,0.000,0.000,0.001,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.483,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.052,0.002,0.007,0.016,0.000,0.251,0.072,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000]])

true_conf=conf*true_cnt
class_f1_weighted_scores, f1_scores_dict = calculate_f1(true_conf)
print(class_f1_weighted_scores)
print(f1_scores_dict)

0.5815922569489625
{0: 0.699774811974041, 1: 0.7766946894287139, 2: 0.5996563573883161, 3: 0.4695710009976721, 4: 0.6967644845748683, 5: 0.8215324927255092, 6: 0.5474452554744527, 7: 0.7108618052014277, 8: 0.4213564213564213, 9: 0.7727546714888487, 10: 0.7688593844296923, 11: 0.7821750321750323, 12: 0.2857142857142857, 13: 0.41666666666666663, 14: 0.0, 15: 0.8247477429633564, 16: 0.6837525707957601, 17: 0.41272776410679485, 18: 0.77231355901716, 19: 0.17638691322901848, 20: 0.0, 21: 0.003861003861003861, 22: 0.33563672260612043, 23: 0.7695035460992907, 24: 0.8340200117716303, 25: 0.6618705035971223, 26: 0.08751139471285323, 27: 0.5188933873144399, 28: 0.5065440396179695, 29: 0.2094240837696335, 30: 0.007858546168958744, 31: 0.8565613490986627, 32: 0.12268314210061783, 33: 0.0}


In [13]:
#xfer_complete_frompred_sj
tmp=stitch_count('../dataset/instruction-complete',['../dataset/val_real_sj.txt'])
true_cnt=ordered_dict_to_list(tmp,34)
conf=np.array([
[0.983,0.005,0.000,0.001,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.001,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.051,0.937,0.001,0.001,0.001,0.001,0.000,0.001,0.000,0.001,0.001,0.000,0.000,0.001,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.108,0.087,0.747,0.017,0.019,0.001,0.001,0.000,0.000,0.000,0.000,0.004,0.000,0.000,0.000,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.009,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.083,0.011,0.005,0.780,0.097,0.003,0.004,0.001,0.000,0.000,0.000,0.007,0.000,0.000,0.000,0.005,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.053,0.009,0.006,0.084,0.813,0.006,0.010,0.000,0.000,0.000,0.002,0.012,0.000,0.000,0.000,0.003,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.070,0.006,0.002,0.004,0.002,0.829,0.000,0.032,0.002,0.000,0.016,0.009,0.000,0.000,0.000,0.021,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.086,0.007,0.012,0.194,0.302,0.000,0.352,0.004,0.000,0.000,0.000,0.011,0.003,0.000,0.000,0.010,0.014,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.041,0.004,0.000,0.009,0.000,0.026,0.000,0.804,0.076,0.009,0.000,0.002,0.000,0.000,0.000,0.017,0.010,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.050,0.007,0.000,0.001,0.000,0.035,0.000,0.063,0.669,0.100,0.012,0.002,0.000,0.000,0.000,0.045,0.013,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.021,0.011,0.000,0.000,0.001,0.012,0.000,0.003,0.039,0.893,0.013,0.001,0.000,0.000,0.000,0.001,0.003,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.026,0.009,0.000,0.000,0.000,0.018,0.000,0.000,0.013,0.010,0.915,0.001,0.000,0.001,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.211,0.005,0.003,0.004,0.006,0.015,0.000,0.002,0.000,0.000,0.000,0.720,0.000,0.003,0.000,0.011,0.019,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.112,0.003,0.014,0.359,0.034,0.000,0.067,0.058,0.000,0.000,0.000,0.052,0.251,0.000,0.000,0.036,0.015,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.157,0.357,0.000,0.001,0.000,0.010,0.000,0.000,0.000,0.000,0.000,0.110,0.000,0.360,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.221,0.014,0.025,0.722,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.013,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.057,0.005,0.002,0.012,0.003,0.029,0.000,0.010,0.019,0.002,0.000,0.008,0.000,0.003,0.000,0.846,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.202,0.005,0.001,0.002,0.000,0.005,0.001,0.011,0.002,0.001,0.000,0.022,0.000,0.000,0.000,0.007,0.736,0.000,0.000,0.000,0.000,0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.042,0.478,0.000,0.001,0.000,0.006,0.000,0.002,0.009,0.000,0.000,0.008,0.000,0.015,0.000,0.000,0.033,0.000,0.000,0.000,0.000,0.000,0.404,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000]])

true_conf=conf*true_cnt
class_f1_weighted_scores, f1_scores_dict = calculate_f1(true_conf)
print(class_f1_weighted_scores)
print(f1_scores_dict)

0.950077597309929
{0: 0.978264695529743, 1: 0.9444925952443758, 2: 0.8250197941409343, 3: 0.7488015340364333, 4: 0.7973421926910299, 5: 0.8380371217899822, 6: 0.48275862068965514, 7: 0.7878787878787878, 8: 0.7307206068268016, 9: 0.8693368351936968, 10: 0.9112508735150244, 11: 0.7798041202296521, 12: 0.5714285714285715, 13: 0.29850746268656714, 14: 0.0, 15: 0.8491812697500718, 16: 0.7814547837483616, 17: 0.0, 18: 0.0, 19: 0.0, 20: 0.0, 21: 0.0, 22: 0.2222222222222222, 23: 0.0, 24: 0.0, 25: 0.0, 26: 0.0, 27: 0.0, 28: 0.0, 29: 0.0, 30: 0.0, 31: 0.0, 32: 0.0, 33: 0.0}


In [14]:
#xfer_complete_frompred_mj
tmp=stitch_count('../dataset/instruction-complete',['../dataset/val_real_mj.txt'])
true_cnt=ordered_dict_to_list(tmp,34)
conf=np.array([
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.547,0.000,0.000,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.374,0.063,0.000,0.000,0.000,0.000,0.000,0.005,0.000,0.001,0.000,0.000,0.001,0.000,0.004,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.020,0.000,0.000,0.287,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.409,0.044,0.000,0.000,0.000,0.000,0.000,0.237,0.000,0.000,0.000,0.000,0.001,0.000,0.002,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.030,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.912,0.033,0.000,0.000,0.000,0.000,0.000,0.011,0.000,0.001,0.003,0.001,0.000,0.000,0.006,0.001,0.000],
[0.000,0.073,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.347,0.567,0.000,0.000,0.000,0.000,0.000,0.006,0.000,0.001,0.000,0.000,0.001,0.000,0.002,0.001,0.000],
[0.000,0.021,0.000,0.000,0.020,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.592,0.209,0.000,0.000,0.000,0.000,0.000,0.154,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.002,0.000],
[0.000,0.060,0.000,0.000,0.153,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.396,0.218,0.000,0.000,0.000,0.000,0.000,0.172,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.033,0.000,0.000,0.172,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.538,0.059,0.000,0.000,0.000,0.000,0.001,0.189,0.000,0.001,0.000,0.000,0.000,0.000,0.005,0.001,0.000],
[0.000,0.045,0.000,0.000,0.032,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.553,0.248,0.000,0.000,0.000,0.000,0.000,0.121,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000],
[0.000,0.005,0.000,0.000,0.032,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.530,0.049,0.000,0.000,0.000,0.000,0.029,0.350,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.006,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.205,0.081,0.000,0.000,0.000,0.000,0.000,0.705,0.000,0.001,0.000,0.000,0.000,0.000,0.002,0.000,0.000],
[0.000,0.013,0.000,0.000,0.023,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.757,0.061,0.000,0.000,0.000,0.000,0.019,0.119,0.005,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.000],
[0.000,0.157,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.457,0.119,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.073,0.002,0.001,0.031,0.000,0.131,0.028,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.662,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.240,0.026,0.000,0.000,0.071,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.734,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.184,0.027,0.001,0.000,0.051,0.000,0.000],
[0.000,0.170,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.393,0.121,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.061,0.002,0.002,0.102,0.000,0.060,0.089,0.000],
[0.000,0.199,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.575,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.128,0.016,0.000,0.000,0.079,0.000,0.000],
[0.000,0.015,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.210,0.007,0.000,0.000,0.000,0.000,0.000,0.005,0.000,0.005,0.003,0.002,0.002,0.000,0.747,0.000,0.000],
[0.000,0.150,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.405,0.172,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.042,0.000,0.000,0.079,0.000,0.019,0.130,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000]])

true_conf=conf*true_cnt
class_f1_weighted_scores, f1_scores_dict = calculate_f1(true_conf)
print(class_f1_weighted_scores)
print(f1_scores_dict)

0.7395524148928677
{0: 0.0, 1: 0.5396232965888379, 2: 0.0, 3: 0.0, 4: 0.3572584997301673, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 0.0, 15: 0.0, 16: 0.0, 17: 0.8744403051903857, 18: 0.5610419026047565, 19: 0.0, 20: 0.0, 21: 0.0, 22: 0.0, 23: 0.05309734513274336, 24: 0.6144590196271821, 25: 0.010309278350515462, 26: 0.11815437688658906, 27: 0.27179890023566383, 28: 0.04554865424430643, 29: 0.14953271028037385, 30: 0.0, 31: 0.7349310094408132, 32: 0.2052545155993432, 33: 0.0}


In [15]:
#xfer_complete_fromtrue_sj
tmp=stitch_count('../dataset/instruction-complete',['../dataset/val_real_sj.txt'])
true_cnt=ordered_dict_to_list(tmp,34)
conf=np.array([
[0.995,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.009,0.989,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.008,0.028,0.919,0.000,0.044,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.001,0.000,0.000,0.888,0.083,0.000,0.012,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.015,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.009,0.000,0.000,0.069,0.905,0.000,0.016,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.982,0.000,0.008,0.005,0.000,0.003,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.056,0.255,0.000,0.660,0.003,0.000,0.000,0.000,0.000,0.023,0.000,0.000,0.002,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.007,0.000,0.013,0.000,0.931,0.023,0.002,0.000,0.000,0.000,0.000,0.000,0.024,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.002,0.000,0.039,0.000,0.033,0.830,0.064,0.000,0.000,0.000,0.000,0.000,0.031,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.011,0.000,0.000,0.024,0.959,0.006,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.009,0.000,0.000,0.000,0.009,0.981,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.089,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.899,0.000,0.000,0.000,0.000,0.010,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.233,0.146,0.000,0.227,0.000,0.000,0.000,0.000,0.000,0.393,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.014,0.398,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.055,0.000,0.508,0.000,0.000,0.022,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.751,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.248,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.002,0.000,0.007,0.000,0.010,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.979,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.083,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.020,0.000,0.000,0.000,0.000,0.897,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.602,0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.392,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000]])

true_conf=conf*true_cnt
class_f1_weighted_scores, f1_scores_dict = calculate_f1(true_conf)
print(class_f1_weighted_scores)
print(f1_scores_dict)

0.9842801406326223
{0: 0.9942019061923641, 1: 0.9926832013808112, 2: 0.9567164179104477, 3: 0.8894447223611806, 4: 0.8692349430276722, 5: 0.9733233607579158, 6: 0.7222222222222223, 7: 0.9267692307692307, 8: 0.8792152053954629, 9: 0.9386138613861386, 10: 0.9845070422535211, 11: 0.9234465086483024, 12: 0.3333333333333333, 13: 0.5172413793103449, 14: 0.0, 15: 0.9691130631907056, 16: 0.9010852713178296, 17: 0.0, 18: 0.0, 19: 0.0, 20: 0.0, 21: 0.0, 22: 0.4230769230769231, 23: 0.0, 24: 0.0, 25: 0.0, 26: 0.0, 27: 0.0, 28: 0.0, 29: 0.0, 30: 0.0, 31: 0.0, 32: 0.0, 33: 0.0}


In [16]:
#xfer_complete_fromtrue_mj
tmp=stitch_count('../dataset/instruction-complete',['../dataset/val_real_mj.txt'])
true_cnt=ordered_dict_to_list(tmp,34)
conf=np.array([
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.849,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.134,0.012,0.002,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.017,0.000,0.000,0.751,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.185,0.001,0.010,0.003,0.010,0.001,0.000,0.022,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.038,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.935,0.018,0.000,0.000,0.001,0.000,0.000,0.005,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000],
[0.000,0.003,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.157,0.836,0.000,0.000,0.000,0.001,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.228,0.121,0.641,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.405,0.000,0.000,0.248,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.090,0.002,0.018,0.203,0.024,0.000,0.000,0.008,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.340,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.265,0.005,0.000,0.007,0.196,0.026,0.009,0.130,0.020,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.000],
[0.000,0.079,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.018,0.344,0.000,0.000,0.003,0.549,0.000,0.000,0.000,0.000,0.000,0.000,0.007,0.000,0.000,0.000,0.000],
[0.000,0.030,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.005,0.097,0.008,0.022,0.025,0.022,0.752,0.000,0.022,0.000,0.000,0.000,0.012,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.038,0.003,0.000,0.001,0.001,0.000,0.000,0.953,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.002,0.000,0.000,0.007,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.023,0.103,0.000,0.000,0.006,0.033,0.003,0.012,0.810,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000],
[0.000,0.220,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.030,0.307,0.000,0.000,0.002,0.002,0.005,0.000,0.008,0.089,0.006,0.004,0.101,0.000,0.184,0.043,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.662,0.331,0.002,0.003,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.004,0.388,0.555,0.001,0.002,0.050,0.000,0.000],
[0.000,0.251,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.049,0.236,0.000,0.000,0.008,0.003,0.003,0.000,0.007,0.069,0.003,0.001,0.201,0.000,0.087,0.080,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.122,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.329,0.150,0.001,0.397,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.001,0.000,0.000,0.000,0.018,0.002,0.008,0.005,0.000,0.954,0.011,0.000],
[0.000,0.185,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.047,0.346,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.050,0.005,0.006,0.085,0.000,0.149,0.125,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000]])

true_conf=conf*true_cnt
class_f1_weighted_scores, f1_scores_dict = calculate_f1(true_conf)
print(class_f1_weighted_scores)
print(f1_scores_dict)

0.8627640102442722
{0: 0.0, 1: 0.7301755447941888, 2: 0.0, 3: 0.0, 4: 0.6996366572466692, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 0.0, 15: 0.0, 16: 0.0, 17: 0.9428595646891512, 18: 0.7827826397570995, 19: 0.7405405405405405, 20: 0.31446540880503143, 21: 0.29906542056074764, 22: 0.6536064113980409, 23: 0.8183361629881154, 24: 0.8922555355440196, 25: 0.8232189973614776, 26: 0.14570685169124023, 27: 0.5610114672155249, 28: 0.5119047619047619, 29: 0.2478688524590164, 30: 0.5333333333333333, 31: 0.900906284234638, 32: 0.20244622522142555, 33: 0.0}


In [17]:
#xfer_complete_frompred_1lyr
tmp=stitch_count('../dataset/instruction-complete',['../dataset/val_real.txt'])
true_cnt=ordered_dict_to_list(tmp,34)
conf=np.array([
[0.962,0.009,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.001,0.006,0.016,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.106,0.853,0.000,0.000,0.001,0.002,0.000,0.000,0.000,0.001,0.001,0.002,0.000,0.000,0.000,0.000,0.001,0.021,0.010,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000],
[0.165,0.690,0.000,0.000,0.001,0.005,0.000,0.000,0.000,0.000,0.002,0.001,0.000,0.000,0.000,0.000,0.002,0.062,0.070,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.100,0.025,0.000,0.005,0.052,0.004,0.000,0.005,0.000,0.001,0.004,0.002,0.000,0.000,0.000,0.000,0.002,0.789,0.010,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.115,0.055,0.000,0.000,0.219,0.005,0.000,0.000,0.000,0.002,0.006,0.010,0.000,0.000,0.000,0.002,0.004,0.554,0.026,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000],
[0.071,0.007,0.000,0.001,0.002,0.596,0.000,0.004,0.000,0.005,0.176,0.012,0.000,0.000,0.000,0.066,0.005,0.054,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.085,0.016,0.000,0.000,0.111,0.000,0.000,0.009,0.000,0.005,0.001,0.003,0.000,0.000,0.000,0.004,0.015,0.631,0.012,0.000,0.000,0.000,0.108,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.067,0.007,0.000,0.001,0.002,0.027,0.000,0.420,0.007,0.004,0.329,0.003,0.000,0.000,0.000,0.033,0.006,0.077,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.017,0.000,0.000],
[0.065,0.007,0.000,0.001,0.008,0.081,0.000,0.366,0.022,0.303,0.008,0.004,0.000,0.000,0.000,0.004,0.017,0.075,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.040,0.000,0.000],
[0.018,0.012,0.000,0.000,0.000,0.024,0.000,0.100,0.142,0.676,0.009,0.001,0.000,0.000,0.000,0.007,0.003,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000],
[0.022,0.017,0.000,0.000,0.001,0.021,0.000,0.008,0.016,0.168,0.734,0.002,0.000,0.000,0.000,0.000,0.005,0.006,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.260,0.084,0.000,0.001,0.003,0.017,0.000,0.001,0.000,0.001,0.001,0.540,0.000,0.000,0.000,0.008,0.018,0.049,0.017,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.014,0.043,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.043,0.000,0.000,0.000,0.000,0.043,0.797,0.043,0.000,0.000,0.000,0.017,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.258,0.279,0.000,0.052,0.019,0.000,0.000,0.000,0.000,0.000,0.000,0.355,0.000,0.000,0.000,0.015,0.015,0.006,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.235,0.047,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.691,0.013,0.000,0.000,0.000,0.012,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.056,0.007,0.000,0.000,0.004,0.238,0.000,0.003,0.002,0.136,0.007,0.012,0.000,0.000,0.000,0.463,0.004,0.067,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.425,0.012,0.000,0.000,0.002,0.004,0.000,0.002,0.000,0.003,0.005,0.044,0.000,0.000,0.000,0.002,0.454,0.046,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.819,0.067,0.000,0.001,0.001,0.002,0.000,0.001,0.001,0.001,0.001,0.002,0.000,0.000,0.000,0.000,0.004,0.073,0.015,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.013,0.000,0.000],
[0.229,0.309,0.000,0.000,0.001,0.001,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.001,0.070,0.384,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000],
[0.130,0.181,0.000,0.000,0.164,0.001,0.000,0.000,0.000,0.000,0.000,0.005,0.000,0.000,0.000,0.002,0.000,0.442,0.074,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.059,0.322,0.000,0.001,0.089,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.486,0.041,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.354,0.119,0.000,0.002,0.025,0.003,0.000,0.001,0.000,0.000,0.000,0.004,0.000,0.000,0.000,0.001,0.002,0.454,0.025,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.008,0.000,0.000],
[0.041,0.692,0.000,0.000,0.002,0.002,0.000,0.003,0.000,0.003,0.008,0.002,0.000,0.000,0.000,0.003,0.062,0.147,0.036,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.252,0.042,0.000,0.000,0.015,0.066,0.000,0.005,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.006,0.002,0.563,0.046,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.108,0.007,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.854,0.025,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000],
[0.338,0.087,0.000,0.001,0.029,0.144,0.000,0.001,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.002,0.009,0.373,0.015,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000],
[0.124,0.251,0.000,0.004,0.016,0.022,0.000,0.003,0.003,0.002,0.001,0.012,0.000,0.000,0.000,0.000,0.010,0.116,0.142,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.291,0.000,0.000],
[0.291,0.000,0.000,0.005,0.004,0.014,0.000,0.073,0.027,0.021,0.032,0.006,0.000,0.000,0.000,0.000,0.028,0.126,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.004,0.015,0.000,0.000,0.354,0.000,0.000],
[0.308,0.000,0.000,0.004,0.003,0.020,0.000,0.096,0.043,0.020,0.036,0.015,0.000,0.000,0.000,0.001,0.034,0.133,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.003,0.016,0.000,0.000,0.268,0.000,0.000],
[0.140,0.250,0.000,0.004,0.013,0.021,0.000,0.001,0.004,0.002,0.003,0.011,0.000,0.000,0.000,0.001,0.011,0.109,0.103,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.325,0.000,0.000],
[0.418,0.304,0.000,0.002,0.002,0.009,0.000,0.029,0.014,0.008,0.012,0.004,0.000,0.000,0.000,0.001,0.016,0.051,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.005,0.000,0.000,0.124,0.000,0.000],
[0.061,0.028,0.000,0.003,0.006,0.008,0.000,0.004,0.005,0.002,0.002,0.003,0.000,0.000,0.000,0.000,0.003,0.135,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.734,0.000,0.000],
[0.175,0.252,0.000,0.002,0.092,0.019,0.000,0.001,0.004,0.000,0.000,0.008,0.000,0.000,0.000,0.001,0.008,0.098,0.111,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.226,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000]])

true_conf=conf*true_cnt
class_f1_weighted_scores, f1_scores_dict = calculate_f1(true_conf)
print(class_f1_weighted_scores)
print(f1_scores_dict)

0.37379271891908766
{0: 0.6269746936658526, 1: 0.6838072093965168, 2: 0.0, 3: 0.008547008547008548, 4: 0.30246646473388145, 5: 0.5330634278002699, 6: 0.0, 7: 0.3429147755925365, 8: 0.029874213836477988, 9: 0.47465656087162483, 10: 0.4793767186067828, 11: 0.6030640668523677, 12: 0.0, 13: 0.0, 14: 0.0, 15: 0.5651414600069856, 16: 0.48612029081295444, 17: 0.12186128456387348, 18: 0.4763617434101557, 19: 0.0, 20: 0.0, 21: 0.0, 22: 0.0, 23: 0.0, 24: 0.0, 25: 0.0, 26: 0.0, 27: 0.008253094910591471, 28: 0.030651340996168577, 29: 0.0, 30: 0.0, 31: 0.5766362883181443, 32: 0.0, 33: 0.0}


In [18]:
#xfer_complete_frompred_3lyr
tmp=stitch_count('../dataset/instruction-complete',['../dataset/val_real.txt'])
true_cnt=ordered_dict_to_list(tmp,34)
conf=np.array([
[0.948,0.007,0.000,0.001,0.001,0.001,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.001,0.004,0.033,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.074,0.832,0.001,0.003,0.002,0.001,0.000,0.000,0.000,0.001,0.001,0.001,0.000,0.001,0.000,0.000,0.001,0.066,0.011,0.000,0.000,0.000,0.001,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.000],
[0.093,0.165,0.578,0.012,0.029,0.001,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.001,0.000,0.079,0.030,0.000,0.000,0.000,0.007,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.096,0.023,0.005,0.624,0.151,0.006,0.016,0.001,0.000,0.000,0.001,0.005,0.000,0.000,0.001,0.008,0.007,0.043,0.002,0.000,0.000,0.003,0.001,0.000,0.003,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.084,0.039,0.005,0.058,0.648,0.005,0.011,0.000,0.000,0.000,0.007,0.011,0.001,0.000,0.000,0.005,0.008,0.073,0.010,0.000,0.002,0.009,0.001,0.001,0.017,0.001,0.002,0.000,0.000,0.000,0.000,0.001,0.000,0.000],
[0.072,0.006,0.001,0.002,0.003,0.816,0.000,0.019,0.003,0.000,0.003,0.013,0.000,0.000,0.000,0.033,0.006,0.016,0.000,0.000,0.000,0.000,0.001,0.003,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000],
[0.061,0.009,0.001,0.093,0.303,0.000,0.427,0.006,0.000,0.001,0.000,0.008,0.003,0.000,0.000,0.019,0.012,0.028,0.000,0.000,0.000,0.007,0.020,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.055,0.005,0.000,0.000,0.005,0.015,0.000,0.800,0.035,0.000,0.000,0.004,0.001,0.000,0.000,0.039,0.020,0.011,0.000,0.000,0.000,0.001,0.005,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.082,0.008,0.000,0.000,0.002,0.041,0.000,0.015,0.759,0.014,0.000,0.004,0.000,0.000,0.000,0.028,0.012,0.029,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.002,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000],
[0.020,0.014,0.000,0.000,0.001,0.007,0.000,0.002,0.011,0.896,0.021,0.001,0.000,0.000,0.000,0.002,0.004,0.019,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.027,0.011,0.000,0.000,0.000,0.014,0.000,0.008,0.004,0.023,0.896,0.001,0.000,0.000,0.000,0.000,0.001,0.009,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.005,0.000,0.000,0.000,0.000,0.000,0.000],
[0.147,0.019,0.004,0.001,0.011,0.017,0.003,0.001,0.001,0.000,0.000,0.743,0.000,0.004,0.000,0.016,0.016,0.015,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.124,0.001,0.000,0.093,0.109,0.000,0.093,0.006,0.000,0.000,0.000,0.042,0.450,0.000,0.000,0.037,0.043,0.001,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.158,0.139,0.000,0.000,0.003,0.016,0.000,0.000,0.000,0.000,0.000,0.141,0.000,0.510,0.000,0.000,0.000,0.034,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.324,0.082,0.218,0.278,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.013,0.000,0.000,0.001,0.000,0.080,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.056,0.008,0.000,0.004,0.005,0.050,0.000,0.007,0.008,0.001,0.000,0.014,0.000,0.000,0.000,0.823,0.007,0.010,0.000,0.000,0.000,0.001,0.005,0.002,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.165,0.007,0.000,0.001,0.004,0.007,0.001,0.006,0.003,0.001,0.000,0.023,0.000,0.000,0.000,0.006,0.750,0.024,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.487,0.035,0.000,0.002,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.434,0.019,0.000,0.000,0.001,0.000,0.000,0.004,0.000,0.001,0.003,0.001,0.001,0.000,0.004,0.001,0.000],
[0.055,0.076,0.002,0.002,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.275,0.574,0.000,0.002,0.000,0.001,0.000,0.002,0.000,0.001,0.000,0.000,0.001,0.000,0.001,0.001,0.000],
[0.130,0.062,0.002,0.008,0.037,0.000,0.000,0.000,0.000,0.000,0.000,0.029,0.000,0.000,0.000,0.003,0.000,0.436,0.050,0.156,0.024,0.001,0.005,0.000,0.057,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.026,0.209,0.000,0.098,0.213,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.229,0.076,0.002,0.120,0.017,0.005,0.001,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.125,0.040,0.001,0.103,0.276,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.318,0.012,0.000,0.001,0.075,0.002,0.002,0.034,0.002,0.000,0.000,0.001,0.000,0.000,0.005,0.000,0.000],
[0.025,0.289,0.000,0.021,0.003,0.007,0.000,0.007,0.005,0.000,0.000,0.013,0.000,0.000,0.000,0.009,0.075,0.105,0.021,0.000,0.005,0.002,0.403,0.001,0.007,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.114,0.044,0.001,0.108,0.124,0.022,0.000,0.015,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.024,0.011,0.276,0.024,0.001,0.001,0.011,0.002,0.214,0.004,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.009,0.004,0.000,0.000,0.007,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.155,0.009,0.000,0.000,0.000,0.000,0.000,0.812,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.000],
[0.193,0.049,0.000,0.099,0.042,0.021,0.001,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.020,0.004,0.289,0.042,0.000,0.001,0.027,0.005,0.002,0.008,0.195,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000],
[0.039,0.135,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.489,0.078,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.089,0.000,0.000,0.082,0.001,0.071,0.013,0.000],
[0.033,0.001,0.000,0.000,0.000,0.004,0.000,0.000,0.002,0.007,0.015,0.000,0.000,0.000,0.000,0.000,0.001,0.574,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.263,0.065,0.000,0.009,0.024,0.000,0.000],
[0.055,0.001,0.000,0.000,0.000,0.001,0.000,0.000,0.001,0.003,0.009,0.001,0.000,0.000,0.000,0.000,0.001,0.682,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.137,0.081,0.000,0.003,0.022,0.000,0.000],
[0.059,0.180,0.001,0.000,0.000,0.002,0.000,0.000,0.000,0.001,0.002,0.000,0.000,0.000,0.000,0.000,0.001,0.436,0.020,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.061,0.000,0.002,0.141,0.000,0.041,0.053,0.000],
[0.133,0.222,0.000,0.001,0.000,0.000,0.000,0.000,0.001,0.002,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.413,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.136,0.044,0.000,0.021,0.023,0.000,0.000],
[0.032,0.021,0.000,0.001,0.003,0.001,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.002,0.000,0.135,0.003,0.000,0.000,0.001,0.000,0.000,0.002,0.000,0.007,0.001,0.001,0.001,0.000,0.788,0.000,0.000],
[0.053,0.209,0.000,0.000,0.001,0.001,0.000,0.000,0.000,0.001,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.463,0.041,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.035,0.000,0.000,0.124,0.000,0.010,0.059,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000]])

true_conf=conf*true_cnt
class_f1_weighted_scores, f1_scores_dict = calculate_f1(true_conf)
print(class_f1_weighted_scores)
print(f1_scores_dict)

0.6391192533946597
{0: 0.7335229094952, 1: 0.7796167247386758, 2: 0.6722268557130943, 3: 0.4860031104199067, 4: 0.6168753726893262, 5: 0.8085742771684945, 6: 0.46527777777777773, 7: 0.8300192431045541, 8: 0.8194878201124298, 9: 0.8960863697705803, 10: 0.8670284938941656, 11: 0.7820807351493272, 12: 0.42857142857142855, 13: 0.36585365853658536, 14: 0.0, 15: 0.8135785007072136, 16: 0.7494217424826524, 17: 0.5495562086440268, 18: 0.6436240749454591, 19: 0.2695417789757413, 20: 0.19601328903654486, 21: 0.12608695652173912, 22: 0.5291902071563089, 23: 0.33495145631067963, 24: 0.829307754535999, 25: 0.3184713375796179, 26: 0.14377406931964057, 27: 0.3003169572107765, 28: 0.12414649286157667, 29: 0.16677115987460817, 30: 0.0412105602060528, 31: 0.8049989226459815, 32: 0.1021963245181533, 33: 0.0}


In [19]:
#xfer_complete_frompred_4lyr
tmp=stitch_count('../dataset/instruction-complete',['../dataset/val_real.txt'])
true_cnt=ordered_dict_to_list(tmp,34)
conf=np.array([
[0.919,0.007,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.001,0.004,0.063,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.059,0.822,0.001,0.001,0.002,0.001,0.000,0.000,0.000,0.001,0.001,0.001,0.000,0.000,0.000,0.000,0.001,0.091,0.011,0.000,0.002,0.000,0.001,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.002,0.000,0.000],
[0.097,0.080,0.705,0.039,0.015,0.001,0.000,0.000,0.000,0.001,0.000,0.004,0.000,0.000,0.003,0.002,0.001,0.045,0.005,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.092,0.018,0.006,0.727,0.075,0.007,0.008,0.000,0.000,0.000,0.001,0.005,0.000,0.000,0.002,0.003,0.006,0.039,0.002,0.000,0.000,0.005,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.074,0.039,0.004,0.036,0.681,0.005,0.007,0.000,0.000,0.000,0.007,0.012,0.001,0.002,0.001,0.003,0.005,0.077,0.007,0.000,0.005,0.021,0.001,0.002,0.004,0.002,0.002,0.000,0.000,0.000,0.001,0.002,0.000,0.000],
[0.068,0.007,0.001,0.002,0.002,0.845,0.000,0.008,0.006,0.001,0.001,0.019,0.000,0.000,0.000,0.025,0.006,0.004,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.075,0.010,0.000,0.067,0.143,0.002,0.594,0.006,0.000,0.002,0.000,0.012,0.021,0.000,0.000,0.013,0.019,0.028,0.000,0.000,0.001,0.001,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.062,0.007,0.000,0.000,0.003,0.013,0.000,0.839,0.014,0.000,0.000,0.002,0.002,0.000,0.000,0.021,0.019,0.007,0.000,0.000,0.006,0.000,0.002,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.065,0.010,0.000,0.000,0.002,0.027,0.000,0.011,0.781,0.026,0.001,0.004,0.000,0.000,0.000,0.013,0.019,0.031,0.000,0.000,0.000,0.000,0.006,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.021,0.015,0.000,0.000,0.000,0.011,0.000,0.000,0.007,0.915,0.016,0.001,0.000,0.000,0.000,0.001,0.003,0.007,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000],
[0.024,0.021,0.000,0.000,0.001,0.009,0.000,0.000,0.001,0.011,0.924,0.000,0.000,0.000,0.000,0.000,0.001,0.006,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000],
[0.114,0.017,0.005,0.001,0.007,0.021,0.000,0.002,0.001,0.000,0.000,0.797,0.000,0.004,0.000,0.011,0.010,0.009,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000],
[0.014,0.000,0.029,0.064,0.021,0.000,0.053,0.018,0.000,0.000,0.000,0.072,0.591,0.000,0.000,0.027,0.097,0.015,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.145,0.152,0.000,0.000,0.005,0.015,0.000,0.000,0.000,0.000,0.000,0.179,0.000,0.475,0.000,0.000,0.000,0.029,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.169,0.081,0.085,0.232,0.041,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.014,0.000,0.252,0.000,0.000,0.125,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.059,0.006,0.000,0.001,0.004,0.017,0.000,0.009,0.007,0.000,0.000,0.014,0.000,0.002,0.000,0.862,0.005,0.005,0.000,0.000,0.001,0.000,0.002,0.003,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.128,0.007,0.000,0.001,0.000,0.006,0.001,0.010,0.004,0.001,0.000,0.017,0.000,0.000,0.000,0.006,0.801,0.014,0.000,0.000,0.000,0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.267,0.022,0.000,0.001,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.668,0.021,0.000,0.000,0.001,0.000,0.000,0.005,0.000,0.002,0.003,0.001,0.000,0.001,0.004,0.000,0.000],
[0.021,0.048,0.001,0.001,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.288,0.624,0.000,0.003,0.000,0.000,0.000,0.003,0.000,0.003,0.000,0.000,0.000,0.000,0.001,0.000,0.000],
[0.117,0.041,0.005,0.002,0.049,0.000,0.001,0.000,0.000,0.000,0.000,0.017,0.000,0.000,0.000,0.002,0.000,0.497,0.022,0.122,0.094,0.002,0.023,0.003,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000],
[0.019,0.103,0.000,0.013,0.118,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.188,0.037,0.001,0.464,0.019,0.026,0.005,0.001,0.002,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.082,0.052,0.001,0.033,0.237,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.001,0.001,0.001,0.400,0.027,0.000,0.007,0.133,0.001,0.005,0.009,0.001,0.001,0.000,0.000,0.000,0.000,0.008,0.000,0.000],
[0.019,0.196,0.000,0.002,0.004,0.000,0.000,0.009,0.003,0.008,0.000,0.006,0.000,0.000,0.000,0.003,0.078,0.085,0.010,0.001,0.009,0.003,0.556,0.004,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.084,0.070,0.001,0.028,0.105,0.006,0.000,0.001,0.001,0.000,0.000,0.001,0.000,0.000,0.000,0.037,0.001,0.317,0.033,0.000,0.029,0.048,0.002,0.221,0.009,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.004,0.005,0.000,0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.179,0.017,0.000,0.000,0.000,0.000,0.000,0.787,0.000,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000],
[0.179,0.076,0.000,0.036,0.045,0.033,0.000,0.007,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.004,0.002,0.349,0.065,0.000,0.011,0.017,0.012,0.002,0.004,0.157,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000],
[0.018,0.074,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.428,0.006,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.300,0.000,0.000,0.032,0.007,0.113,0.019,0.000],
[0.034,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.003,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.652,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.208,0.065,0.000,0.015,0.018,0.000,0.000],
[0.010,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.737,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.147,0.070,0.000,0.011,0.019,0.000,0.000],
[0.018,0.121,0.001,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.530,0.012,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.175,0.000,0.000,0.090,0.000,0.014,0.038,0.000],
[0.102,0.088,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.001,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.525,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.137,0.028,0.000,0.106,0.007,0.000,0.000],
[0.008,0.023,0.000,0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.126,0.004,0.000,0.001,0.001,0.000,0.000,0.008,0.000,0.005,0.002,0.000,0.000,0.000,0.814,0.000,0.000],
[0.029,0.102,0.001,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.616,0.024,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.097,0.000,0.000,0.048,0.000,0.009,0.072,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000]])

true_conf=conf*true_cnt
class_f1_weighted_scores, f1_scores_dict = calculate_f1(true_conf)
print(class_f1_weighted_scores)
print(f1_scores_dict)

0.7409575259315971
{0: 0.8071134235777763, 1: 0.812553688794774, 2: 0.7699530516431926, 3: 0.6845322049835448, 4: 0.6796936063779897, 5: 0.8427389014296464, 6: 0.6573426573426573, 7: 0.8605830164765526, 8: 0.8458646616541353, 9: 0.9088471849865952, 10: 0.9095928226363008, 11: 0.8173968660057565, 12: 0.4444444444444444, 13: 0.43750000000000006, 14: 0.33962264150943394, 15: 0.8588537211291702, 16: 0.7987072945521698, 17: 0.7205774936654022, 18: 0.6790948723271746, 19: 0.21666666666666667, 20: 0.4935344827586207, 21: 0.20797576981322566, 22: 0.6598639455782314, 23: 0.33890214797136037, 24: 0.8146152945309755, 25: 0.2569593147751606, 26: 0.3629938965628012, 27: 0.24400162667751119, 28: 0.11089866156787763, 29: 0.1441899915182358, 30: 0.1744377399890291, 31: 0.8200169995750105, 32: 0.13021077283372368, 33: 0.0}


In [20]:
#xfer_complete_frompred_5lyr
tmp=stitch_count('../dataset/instruction-complete',['../dataset/val_real.txt'])
true_cnt=ordered_dict_to_list(tmp,34)
conf=np.array([
[0.936,0.008,0.000,0.001,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.001,0.003,0.046,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.049,0.807,0.001,0.001,0.002,0.001,0.000,0.001,0.000,0.001,0.001,0.001,0.000,0.000,0.000,0.000,0.001,0.116,0.012,0.000,0.001,0.000,0.001,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.002,0.000,0.000],
[0.098,0.101,0.740,0.021,0.010,0.001,0.000,0.000,0.000,0.001,0.000,0.004,0.000,0.000,0.002,0.000,0.001,0.016,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.097,0.022,0.006,0.761,0.048,0.006,0.010,0.000,0.001,0.000,0.001,0.007,0.000,0.000,0.001,0.004,0.006,0.024,0.004,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.064,0.038,0.005,0.022,0.683,0.005,0.005,0.000,0.000,0.001,0.006,0.013,0.000,0.001,0.001,0.004,0.005,0.096,0.012,0.000,0.007,0.020,0.001,0.003,0.002,0.001,0.001,0.000,0.000,0.001,0.000,0.001,0.000,0.000],
[0.076,0.009,0.001,0.002,0.002,0.851,0.000,0.005,0.005,0.000,0.000,0.015,0.000,0.000,0.000,0.015,0.006,0.006,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000],
[0.058,0.011,0.001,0.065,0.073,0.001,0.706,0.001,0.000,0.001,0.000,0.006,0.007,0.000,0.000,0.010,0.026,0.026,0.000,0.000,0.000,0.000,0.005,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.057,0.002,0.000,0.001,0.001,0.009,0.000,0.862,0.015,0.001,0.000,0.005,0.002,0.000,0.000,0.016,0.022,0.003,0.000,0.000,0.000,0.001,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.056,0.005,0.000,0.000,0.001,0.012,0.000,0.007,0.857,0.003,0.001,0.003,0.000,0.000,0.000,0.011,0.018,0.024,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.021,0.015,0.000,0.000,0.001,0.004,0.000,0.000,0.009,0.935,0.007,0.001,0.000,0.000,0.000,0.001,0.003,0.002,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.032,0.013,0.000,0.000,0.001,0.005,0.000,0.000,0.001,0.011,0.932,0.001,0.000,0.000,0.000,0.000,0.001,0.002,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.114,0.015,0.006,0.002,0.006,0.016,0.000,0.002,0.000,0.000,0.000,0.806,0.000,0.003,0.000,0.011,0.010,0.007,0.001,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.076,0.023,0.064,0.162,0.031,0.000,0.066,0.000,0.000,0.000,0.000,0.109,0.401,0.000,0.000,0.004,0.064,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.138,0.112,0.000,0.000,0.000,0.014,0.000,0.000,0.000,0.000,0.000,0.209,0.000,0.495,0.000,0.000,0.000,0.032,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.270,0.072,0.059,0.136,0.025,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.269,0.002,0.002,0.164,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.058,0.010,0.001,0.003,0.004,0.019,0.001,0.010,0.002,0.000,0.000,0.016,0.000,0.000,0.000,0.859,0.006,0.004,0.000,0.000,0.000,0.000,0.001,0.003,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.110,0.008,0.000,0.000,0.002,0.003,0.001,0.009,0.003,0.000,0.000,0.016,0.000,0.000,0.000,0.006,0.828,0.011,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.180,0.023,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.752,0.022,0.000,0.001,0.001,0.000,0.000,0.004,0.000,0.004,0.004,0.001,0.001,0.002,0.003,0.000,0.000],
[0.013,0.064,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.350,0.558,0.000,0.001,0.000,0.000,0.000,0.002,0.000,0.004,0.000,0.000,0.001,0.000,0.002,0.001,0.000],
[0.027,0.026,0.000,0.001,0.019,0.000,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.001,0.000,0.398,0.045,0.379,0.081,0.002,0.010,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.006,0.000,0.000],
[0.004,0.082,0.000,0.003,0.070,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.189,0.017,0.003,0.588,0.017,0.020,0.004,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.051,0.082,0.000,0.011,0.162,0.000,0.000,0.000,0.001,0.000,0.000,0.001,0.000,0.000,0.000,0.001,0.001,0.448,0.056,0.000,0.014,0.151,0.006,0.003,0.002,0.002,0.001,0.000,0.000,0.000,0.000,0.009,0.000,0.000],
[0.012,0.174,0.000,0.001,0.004,0.002,0.000,0.009,0.002,0.000,0.000,0.004,0.000,0.000,0.000,0.000,0.076,0.045,0.008,0.002,0.010,0.002,0.643,0.003,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.039,0.033,0.000,0.078,0.121,0.001,0.000,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.006,0.000,0.338,0.051,0.000,0.005,0.041,0.011,0.263,0.001,0.006,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.004,0.002,0.000,0.001,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.136,0.009,0.000,0.000,0.000,0.000,0.000,0.843,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.000],
[0.069,0.045,0.000,0.010,0.046,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.506,0.054,0.000,0.007,0.019,0.010,0.006,0.001,0.222,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.000],
[0.004,0.088,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.497,0.007,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.272,0.000,0.000,0.055,0.004,0.051,0.020,0.000],
[0.012,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.574,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.333,0.054,0.000,0.017,0.007,0.000,0.000],
[0.009,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.712,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.003,0.163,0.100,0.000,0.008,0.003,0.000,0.000],
[0.002,0.101,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.488,0.014,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.201,0.000,0.000,0.149,0.001,0.009,0.034,0.000],
[0.050,0.032,0.000,0.000,0.001,0.000,0.000,0.000,0.001,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.604,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.147,0.048,0.000,0.105,0.009,0.000,0.000],
[0.002,0.025,0.000,0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.133,0.006,0.000,0.000,0.001,0.000,0.000,0.001,0.000,0.007,0.000,0.000,0.000,0.000,0.818,0.000,0.000],
[0.013,0.078,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.583,0.044,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.143,0.000,0.000,0.059,0.000,0.003,0.074,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000]])

true_conf=conf*true_cnt
class_f1_weighted_scores, f1_scores_dict = calculate_f1(true_conf)
print(class_f1_weighted_scores)
print(f1_scores_dict)

0.7806578648192927
{0: 0.8578200972447325, 1: 0.7988477004021045, 2: 0.8034188034188033, 3: 0.744504152418173, 4: 0.7194458189015339, 5: 0.8601931875953228, 6: 0.744186046511628, 7: 0.8674548848786559, 8: 0.8948004836759371, 9: 0.9371196754563895, 10: 0.926829268292683, 11: 0.8362989323843416, 12: 0.4444444444444445, 13: 0.509090909090909, 14: 0.3846153846153846, 15: 0.8663780663780664, 16: 0.8400125234815278, 17: 0.7768327922312861, 18: 0.6236834250138586, 19: 0.5485327313769752, 20: 0.5576923076923077, 21: 0.2342342342342342, 22: 0.7360655737704918, 23: 0.38799076212471123, 24: 0.864971554626727, 25: 0.34907597535934287, 26: 0.3006459189665297, 27: 0.3465703971119134, 28: 0.15318097591105623, 29: 0.19704433497536944, 30: 0.16501305483028722, 31: 0.8457093010576818, 32: 0.13221601489757911, 33: 0.0}


In [21]:
#xfer_complete_frompred_residual
tmp=stitch_count('../dataset/instruction-complete',['../dataset/val_real.txt'])
true_cnt=ordered_dict_to_list(tmp,34)
conf=np.array([
[0.967,0.007,0.001,0.001,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.001,0.004,0.014,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.052,0.810,0.001,0.001,0.003,0.001,0.000,0.000,0.000,0.001,0.001,0.001,0.000,0.000,0.000,0.000,0.001,0.107,0.016,0.000,0.000,0.001,0.001,0.000,0.001,0.000,0.001,0.000,0.000,0.000,0.000,0.001,0.000,0.000],
[0.171,0.112,0.603,0.069,0.022,0.002,0.000,0.000,0.000,0.000,0.001,0.002,0.000,0.000,0.000,0.005,0.001,0.006,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.133,0.016,0.010,0.727,0.070,0.007,0.008,0.001,0.000,0.002,0.001,0.005,0.000,0.000,0.000,0.003,0.009,0.007,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.074,0.035,0.006,0.052,0.668,0.007,0.005,0.000,0.000,0.000,0.006,0.014,0.000,0.001,0.000,0.003,0.007,0.093,0.010,0.000,0.000,0.012,0.001,0.001,0.001,0.001,0.001,0.000,0.000,0.000,0.000,0.001,0.000,0.000],
[0.077,0.007,0.001,0.003,0.002,0.828,0.000,0.016,0.012,0.002,0.001,0.016,0.000,0.000,0.000,0.027,0.008,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.083,0.013,0.017,0.137,0.226,0.001,0.375,0.015,0.000,0.000,0.000,0.035,0.003,0.002,0.000,0.021,0.035,0.000,0.000,0.000,0.000,0.000,0.038,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.061,0.004,0.000,0.007,0.002,0.014,0.003,0.849,0.007,0.000,0.000,0.004,0.001,0.000,0.000,0.028,0.019,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.079,0.007,0.000,0.001,0.001,0.035,0.000,0.032,0.774,0.022,0.001,0.003,0.000,0.000,0.000,0.023,0.020,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.025,0.012,0.000,0.000,0.001,0.024,0.000,0.004,0.012,0.894,0.021,0.002,0.000,0.000,0.000,0.001,0.003,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.028,0.010,0.000,0.000,0.001,0.021,0.000,0.008,0.002,0.021,0.906,0.001,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.157,0.016,0.005,0.002,0.008,0.023,0.002,0.002,0.001,0.000,0.000,0.749,0.000,0.003,0.000,0.012,0.017,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.047,0.000,0.027,0.286,0.230,0.000,0.158,0.014,0.000,0.000,0.000,0.061,0.135,0.000,0.000,0.000,0.042,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.148,0.215,0.000,0.000,0.035,0.016,0.000,0.000,0.000,0.000,0.000,0.133,0.000,0.449,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.334,0.102,0.179,0.363,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.019,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.062,0.006,0.001,0.006,0.004,0.023,0.000,0.016,0.007,0.000,0.000,0.019,0.000,0.000,0.000,0.850,0.007,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.149,0.007,0.001,0.001,0.000,0.009,0.002,0.009,0.008,0.000,0.000,0.025,0.000,0.000,0.000,0.005,0.781,0.000,0.000,0.000,0.000,0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.012,0.014,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.927,0.024,0.000,0.001,0.001,0.000,0.000,0.004,0.000,0.003,0.004,0.002,0.001,0.000,0.003,0.001,0.000],
[0.003,0.050,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.290,0.642,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.002,0.000,0.000,0.001,0.000,0.002,0.004,0.000],
[0.000,0.009,0.000,0.000,0.010,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.514,0.013,0.235,0.212,0.000,0.007,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.022,0.000,0.000,0.032,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.443,0.018,0.005,0.472,0.000,0.007,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.053,0.000,0.000,0.258,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.529,0.050,0.000,0.001,0.085,0.000,0.001,0.013,0.000,0.000,0.000,0.000,0.000,0.000,0.009,0.000,0.000],
[0.008,0.254,0.000,0.000,0.002,0.000,0.000,0.007,0.002,0.000,0.000,0.003,0.000,0.000,0.000,0.005,0.081,0.116,0.005,0.002,0.017,0.000,0.495,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.001,0.033,0.000,0.001,0.248,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.456,0.069,0.000,0.001,0.042,0.023,0.115,0.001,0.010,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000],
[0.002,0.002,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.098,0.005,0.000,0.000,0.000,0.000,0.000,0.890,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.000],
[0.002,0.052,0.000,0.000,0.054,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.655,0.033,0.000,0.000,0.064,0.010,0.007,0.012,0.111,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000],
[0.003,0.036,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.604,0.009,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.271,0.000,0.000,0.056,0.000,0.002,0.018,0.000],
[0.019,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.614,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.287,0.051,0.000,0.025,0.001,0.000,0.000],
[0.008,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.735,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.140,0.113,0.000,0.001,0.000,0.001,0.000],
[0.000,0.016,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.647,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.161,0.000,0.000,0.146,0.000,0.000,0.025,0.000],
[0.020,0.010,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.809,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.001,0.098,0.019,0.000,0.039,0.000,0.000,0.000],
[0.000,0.021,0.000,0.000,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.126,0.004,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.842,0.000,0.000],
[0.005,0.037,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.631,0.015,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.045,0.000,0.000,0.051,0.000,0.001,0.214,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000]])

true_conf=conf*true_cnt
class_f1_weighted_scores, f1_scores_dict = calculate_f1(true_conf)
print(class_f1_weighted_scores)
print(f1_scores_dict)

0.859238274353198
{0: 0.958777856317021, 1: 0.8264428727550158, 2: 0.6522075910147171, 3: 0.6829268292682927, 4: 0.660473757547608, 5: 0.8236177132849636, 6: 0.4609375, 7: 0.8398533007334964, 8: 0.828482972136223, 9: 0.9012933968686182, 10: 0.9028651292802237, 11: 0.7853522968775544, 12: 0.25, 13: 0.4814814814814815, 14: 0.0, 15: 0.8495423340961098, 16: 0.7940991839296925, 17: 0.8893717685642345, 18: 0.6807718758849468, 19: 0.38035264483627207, 20: 0.474059003051882, 21: 0.13909378292939933, 22: 0.6234234234234234, 23: 0.20273972602739723, 24: 0.8879105993904504, 25: 0.19724770642201836, 26: 0.3388704318936877, 27: 0.31846153846153846, 28: 0.16548463356973997, 29: 0.19658119658119658, 30: 0.07268170426065163, 31: 0.8740967812568426, 32: 0.32250489236790614, 33: 0.0}


In [22]:
#xfer_complete_frompred_unet
tmp=stitch_count('../dataset/instruction-complete',['../dataset/val_real.txt'])
true_cnt=ordered_dict_to_list(tmp,34)
conf=np.array([
[0.957,0.008,0.001,0.000,0.001,0.001,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.001,0.007,0.020,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.045,0.810,0.001,0.001,0.004,0.001,0.000,0.000,0.000,0.001,0.001,0.001,0.000,0.000,0.000,0.000,0.001,0.109,0.018,0.000,0.000,0.001,0.002,0.000,0.001,0.000,0.001,0.000,0.000,0.000,0.000,0.001,0.001,0.000],
[0.128,0.210,0.578,0.020,0.018,0.002,0.000,0.001,0.000,0.000,0.001,0.009,0.000,0.000,0.000,0.002,0.001,0.007,0.005,0.000,0.000,0.000,0.019,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.116,0.027,0.009,0.656,0.122,0.006,0.009,0.007,0.000,0.000,0.001,0.008,0.000,0.000,0.000,0.012,0.007,0.014,0.002,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.063,0.034,0.004,0.045,0.675,0.004,0.005,0.001,0.000,0.000,0.006,0.013,0.000,0.000,0.000,0.007,0.004,0.104,0.021,0.000,0.001,0.006,0.001,0.001,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000],
[0.065,0.007,0.001,0.002,0.003,0.782,0.000,0.040,0.007,0.000,0.005,0.013,0.000,0.000,0.000,0.065,0.006,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.088,0.006,0.006,0.121,0.390,0.001,0.292,0.027,0.000,0.000,0.000,0.016,0.000,0.000,0.000,0.015,0.008,0.021,0.000,0.000,0.000,0.000,0.010,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.061,0.005,0.000,0.002,0.003,0.033,0.000,0.788,0.053,0.001,0.003,0.004,0.000,0.000,0.000,0.034,0.013,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.055,0.004,0.000,0.000,0.001,0.065,0.002,0.078,0.665,0.039,0.010,0.005,0.000,0.000,0.000,0.054,0.018,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.028,0.014,0.000,0.000,0.001,0.013,0.000,0.000,0.030,0.867,0.040,0.001,0.000,0.000,0.000,0.001,0.004,0.001,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.029,0.015,0.000,0.000,0.001,0.018,0.000,0.000,0.003,0.038,0.885,0.001,0.000,0.000,0.000,0.000,0.002,0.001,0.000,0.000,0.000,0.000,0.008,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.167,0.018,0.004,0.001,0.014,0.027,0.001,0.001,0.002,0.000,0.000,0.733,0.000,0.000,0.000,0.007,0.020,0.002,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.040,0.001,0.000,0.215,0.387,0.000,0.070,0.014,0.001,0.000,0.000,0.069,0.154,0.000,0.000,0.000,0.035,0.000,0.000,0.000,0.000,0.000,0.015,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.236,0.280,0.000,0.000,0.076,0.013,0.000,0.000,0.000,0.000,0.000,0.199,0.000,0.160,0.000,0.007,0.010,0.019,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.089,0.008,0.293,0.516,0.037,0.000,0.000,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.052,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.056,0.005,0.001,0.007,0.008,0.079,0.000,0.011,0.025,0.002,0.000,0.015,0.000,0.000,0.000,0.779,0.006,0.001,0.000,0.000,0.001,0.000,0.001,0.002,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.244,0.004,0.001,0.000,0.003,0.010,0.001,0.007,0.006,0.001,0.000,0.039,0.000,0.000,0.000,0.005,0.671,0.002,0.000,0.000,0.000,0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.015,0.023,0.000,0.000,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.913,0.027,0.000,0.000,0.001,0.000,0.000,0.004,0.000,0.001,0.004,0.001,0.000,0.001,0.003,0.001,0.000],
[0.005,0.043,0.000,0.000,0.006,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.369,0.569,0.000,0.000,0.001,0.000,0.000,0.003,0.000,0.001,0.000,0.000,0.001,0.000,0.001,0.001,0.000],
[0.018,0.019,0.000,0.005,0.052,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.628,0.024,0.161,0.059,0.004,0.028,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.015,0.045,0.000,0.010,0.042,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.397,0.033,0.005,0.436,0.005,0.011,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.004,0.041,0.000,0.000,0.209,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.548,0.032,0.000,0.010,0.067,0.002,0.001,0.075,0.001,0.003,0.000,0.000,0.000,0.000,0.005,0.001,0.000],
[0.017,0.383,0.000,0.003,0.002,0.007,0.000,0.001,0.003,0.000,0.000,0.012,0.000,0.000,0.000,0.003,0.075,0.086,0.005,0.000,0.005,0.000,0.397,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.010,0.095,0.000,0.000,0.192,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.469,0.042,0.000,0.006,0.025,0.002,0.136,0.018,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.001,0.004,0.000,0.000,0.013,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.137,0.007,0.000,0.000,0.000,0.000,0.000,0.832,0.000,0.000,0.000,0.000,0.000,0.000,0.006,0.000,0.000],
[0.009,0.115,0.001,0.000,0.096,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.550,0.037,0.000,0.001,0.049,0.006,0.070,0.016,0.049,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000],
[0.003,0.047,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.708,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.152,0.000,0.000,0.037,0.000,0.009,0.041,0.000],
[0.019,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.644,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.304,0.029,0.000,0.000,0.000,0.000,0.000],
[0.007,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.776,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.160,0.050,0.000,0.000,0.000,0.000,0.000],
[0.000,0.075,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.729,0.012,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.086,0.000,0.000,0.071,0.000,0.001,0.024,0.000],
[0.018,0.021,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.823,0.001,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.001,0.075,0.009,0.000,0.049,0.000,0.000,0.000],
[0.000,0.020,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.166,0.006,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.002,0.000,0.000,0.000,0.000,0.801,0.000,0.000],
[0.007,0.089,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.537,0.019,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.070,0.000,0.000,0.092,0.000,0.002,0.183,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000]])

true_conf=conf*true_cnt
class_f1_weighted_scores, f1_scores_dict = calculate_f1(true_conf)
print(class_f1_weighted_scores)
print(f1_scores_dict)

0.8385804846145737
{0: 0.9514620718453642, 1: 0.803802418627038, 2: 0.6346456692913385, 3: 0.687958115183246, 4: 0.6172132321351854, 5: 0.7688504326328801, 6: 0.3881856540084388, 7: 0.7697885196374622, 8: 0.7223618090452262, 9: 0.8687415426251691, 10: 0.8679504814305365, 11: 0.7699361388570494, 12: 0.4, 13: 0.29411764705882354, 14: 0.0, 15: 0.782183908045977, 16: 0.6951239008792965, 17: 0.8733026014865638, 18: 0.6163064589445846, 19: 0.2765100671140939, 20: 0.5584415584415584, 21: 0.11231101511879049, 22: 0.5031789282470481, 23: 0.2194513715710723, 24: 0.8420932096644911, 25: 0.0920096852300242, 26: 0.22413793103448276, 27: 0.3349904397705545, 28: 0.0848153214774282, 29: 0.1056782334384858, 30: 0.0879382055852644, 31: 0.8484306936222495, 32: 0.27903289734443126, 33: 0.0}


In [31]:
#xfer_complete_frompred_residual_sj
tmp=stitch_count('../dataset/instruction-complete',['../dataset/val_real_sj.txt'])
true_cnt=ordered_dict_to_list(tmp,34)
conf=np.array([
[0.988,0.005,0.000,0.001,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.001,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.043,0.949,0.002,0.001,0.000,0.001,0.000,0.001,0.000,0.001,0.001,0.001,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.097,0.043,0.827,0.013,0.009,0.002,0.000,0.000,0.000,0.001,0.000,0.003,0.000,0.000,0.002,0.001,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.081,0.006,0.004,0.877,0.016,0.001,0.002,0.001,0.000,0.000,0.000,0.003,0.000,0.000,0.001,0.002,0.004,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.026,0.008,0.003,0.011,0.919,0.006,0.003,0.000,0.000,0.000,0.001,0.014,0.000,0.000,0.001,0.003,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.065,0.004,0.001,0.003,0.002,0.880,0.001,0.002,0.004,0.000,0.000,0.015,0.000,0.003,0.000,0.013,0.009,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.057,0.003,0.000,0.046,0.045,0.014,0.783,0.003,0.000,0.000,0.000,0.009,0.000,0.000,0.000,0.007,0.029,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.030,0.004,0.000,0.002,0.000,0.001,0.000,0.919,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.018,0.020,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.045,0.005,0.000,0.001,0.001,0.025,0.000,0.007,0.885,0.001,0.000,0.002,0.000,0.000,0.000,0.004,0.020,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.027,0.012,0.000,0.000,0.001,0.001,0.000,0.000,0.004,0.946,0.004,0.001,0.000,0.000,0.000,0.001,0.002,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.028,0.012,0.000,0.000,0.000,0.007,0.000,0.000,0.001,0.005,0.945,0.001,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.073,0.004,0.002,0.002,0.003,0.014,0.001,0.001,0.000,0.000,0.000,0.880,0.000,0.004,0.000,0.010,0.006,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.038,0.000,0.000,0.102,0.031,0.000,0.067,0.000,0.000,0.000,0.000,0.038,0.614,0.000,0.000,0.000,0.109,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.075,0.068,0.000,0.000,0.001,0.015,0.000,0.000,0.000,0.000,0.000,0.164,0.000,0.677,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.266,0.031,0.063,0.474,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.156,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.041,0.004,0.001,0.004,0.002,0.013,0.001,0.006,0.000,0.000,0.000,0.012,0.000,0.003,0.000,0.903,0.011,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.079,0.003,0.000,0.001,0.001,0.004,0.000,0.008,0.005,0.000,0.000,0.010,0.000,0.000,0.000,0.007,0.877,0.000,0.000,0.000,0.000,0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.002,0.060,0.000,0.000,0.000,0.001,0.001,0.007,0.001,0.000,0.000,0.000,0.000,0.002,0.000,0.002,0.040,0.000,0.000,0.000,0.000,0.000,0.883,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000]])

true_conf=conf*true_cnt
class_f1_weighted_scores, f1_scores_dict = calculate_f1(true_conf)
print(class_f1_weighted_scores)
print(f1_scores_dict)

0.9696970100615798
{0: 0.9854152197638655, 1: 0.9523916713562183, 2: 0.8742424242424243, 3: 0.8565853658536585, 4: 0.9302059496567505, 5: 0.8826646479939441, 6: 0.8493150684931507, 7: 0.9206939281288724, 8: 0.9233554616777309, 9: 0.9589603283173734, 10: 0.9587482219061167, 11: 0.8906422597443585, 12: 0.8, 13: 0.547945205479452, 14: 0.26666666666666666, 15: 0.8955732122587968, 16: 0.8910145836600282, 17: 0.0, 18: 0.0, 19: 0.0, 20: 0.0, 21: 0.0, 22: 0.5714285714285714, 23: 0.0, 24: 0.0, 25: 0.0, 26: 0.0, 27: 0.0, 28: 0.0, 29: 0.0, 30: 0.0, 31: 0.0, 32: 0.0, 33: 0.0}


In [32]:
true_cnt

array([[90474],
       [13375],
       [  698],
       [ 1001],
       [  885],
       [ 1988],
       [  158],
       [  809],
       [  864],
       [  741],
       [  713],
       [ 3207],
       [    6],
       [   29],
       [   36],
       [ 1747],
       [ 3240],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0],
       [   29],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0]])

In [24]:
#xfer_complete_frompred_unet_sj
tmp=stitch_count('../dataset/instruction-complete',['../dataset/val_real_sj.txt'])
true_cnt=ordered_dict_to_list(tmp,34)
conf=np.array([
[0.985,0.005,0.000,0.001,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.001,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.046,0.944,0.002,0.001,0.000,0.001,0.000,0.000,0.000,0.001,0.001,0.001,0.000,0.001,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.076,0.075,0.818,0.007,0.010,0.000,0.000,0.000,0.000,0.000,0.000,0.006,0.000,0.000,0.001,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.074,0.010,0.006,0.820,0.064,0.002,0.003,0.000,0.000,0.000,0.000,0.005,0.000,0.000,0.000,0.009,0.004,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.035,0.010,0.007,0.029,0.879,0.006,0.009,0.000,0.000,0.000,0.001,0.014,0.000,0.000,0.002,0.007,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.070,0.007,0.001,0.005,0.002,0.871,0.000,0.003,0.007,0.000,0.001,0.009,0.000,0.000,0.000,0.019,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.081,0.003,0.004,0.104,0.327,0.000,0.414,0.003,0.000,0.000,0.000,0.013,0.009,0.000,0.000,0.028,0.013,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.042,0.006,0.000,0.001,0.001,0.025,0.000,0.880,0.004,0.000,0.000,0.004,0.000,0.001,0.000,0.020,0.014,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.047,0.007,0.000,0.001,0.001,0.029,0.001,0.007,0.879,0.003,0.001,0.002,0.000,0.000,0.000,0.003,0.016,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.030,0.014,0.000,0.000,0.001,0.007,0.000,0.000,0.016,0.911,0.016,0.000,0.000,0.000,0.000,0.001,0.002,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.033,0.013,0.000,0.000,0.001,0.006,0.000,0.000,0.002,0.015,0.924,0.001,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.108,0.007,0.002,0.001,0.006,0.016,0.001,0.001,0.001,0.000,0.000,0.829,0.000,0.004,0.000,0.012,0.014,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.040,0.012,0.000,0.053,0.273,0.000,0.317,0.059,0.000,0.000,0.000,0.035,0.153,0.000,0.000,0.009,0.050,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.075,0.202,0.000,0.000,0.000,0.005,0.000,0.000,0.000,0.000,0.000,0.145,0.000,0.540,0.000,0.023,0.000,0.000,0.000,0.000,0.000,0.000,0.010,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.197,0.044,0.327,0.297,0.071,0.000,0.042,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.004,0.016,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.059,0.004,0.001,0.004,0.002,0.030,0.000,0.008,0.006,0.003,0.003,0.009,0.000,0.003,0.000,0.863,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.118,0.005,0.001,0.000,0.001,0.007,0.001,0.008,0.006,0.000,0.001,0.030,0.000,0.000,0.000,0.005,0.814,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.020,0.289,0.000,0.000,0.000,0.000,0.000,0.008,0.003,0.000,0.000,0.001,0.000,0.000,0.000,0.001,0.044,0.000,0.000,0.000,0.000,0.000,0.634,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000]])

true_conf=conf*true_cnt
class_f1_weighted_scores, f1_scores_dict = calculate_f1(true_conf)
print(class_f1_weighted_scores)
print(f1_scores_dict)

0.9622653533117778
{0: 0.9831159160484294, 1: 0.9478978978978979, 2: 0.8567141785446362, 3: 0.8197703444832751, 4: 0.8530701754385964, 5: 0.8636250311642981, 6: 0.5349794238683128, 7: 0.9041269841269841, 8: 0.9003558718861211, 9: 0.9329647546648239, 10: 0.9347517730496454, 11: 0.8538856775850995, 12: 0.28571428571428575, 13: 0.4155844155844156, 14: 0.0, 15: 0.8671650373778034, 16: 0.8427612655800575, 17: 0.0, 18: 0.0, 19: 0.0, 20: 0.0, 21: 0.0, 22: 0.3829787234042553, 23: 0.0, 24: 0.0, 25: 0.0, 26: 0.0, 27: 0.0, 28: 0.0, 29: 0.0, 30: 0.0, 31: 0.0, 32: 0.0, 33: 0.0}


In [33]:
#xfer_complete_frompred_residual_mj
tmp=stitch_count('../dataset/instruction-complete',['../dataset/val_real_mj.txt'])
true_cnt=ordered_dict_to_list(tmp,34)
conf=np.array([
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.830,0.000,0.000,0.006,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.115,0.037,0.000,0.000,0.003,0.000,0.000,0.002,0.000,0.001,0.000,0.000,0.001,0.000,0.004,0.001,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.027,0.000,0.000,0.832,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.095,0.019,0.000,0.000,0.005,0.000,0.009,0.006,0.006,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.013,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.955,0.012,0.000,0.001,0.002,0.000,0.000,0.001,0.000,0.003,0.002,0.003,0.001,0.002,0.001,0.001,0.000],
[0.000,0.040,0.000,0.000,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.145,0.798,0.000,0.000,0.002,0.000,0.000,0.003,0.001,0.002,0.000,0.000,0.001,0.000,0.003,0.000,0.000],
[0.000,0.024,0.000,0.000,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.470,0.010,0.355,0.086,0.005,0.042,0.002,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.024,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.240,0.009,0.021,0.678,0.004,0.019,0.002,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.023,0.000,0.000,0.012,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.166,0.017,0.000,0.000,0.771,0.000,0.002,0.000,0.001,0.001,0.000,0.000,0.000,0.000,0.007,0.000,0.000],
[0.000,0.024,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.360,0.011,0.011,0.107,0.006,0.474,0.002,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.012,0.000,0.000,0.282,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.168,0.009,0.000,0.000,0.011,0.000,0.471,0.006,0.041,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000],
[0.000,0.002,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.024,0.007,0.000,0.000,0.000,0.000,0.000,0.964,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000],
[0.000,0.030,0.000,0.000,0.143,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.289,0.037,0.000,0.000,0.007,0.000,0.017,0.006,0.467,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000],
[0.000,0.010,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.222,0.008,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.697,0.000,0.000,0.038,0.000,0.000,0.025,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.239,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.644,0.016,0.003,0.096,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.192,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.806,0.000,0.001,0.000,0.000,0.000],
[0.000,0.014,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.317,0.007,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.117,0.000,0.000,0.530,0.000,0.000,0.014,0.000],
[0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.367,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.192,0.001,0.001,0.433,0.000,0.001,0.000],
[0.000,0.023,0.000,0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.038,0.004,0.000,0.000,0.003,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.924,0.000,0.000],
[0.000,0.018,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.149,0.019,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.044,0.000,0.000,0.004,0.000,0.000,0.764,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000]])

true_conf=conf*true_cnt
class_f1_weighted_scores, f1_scores_dict = calculate_f1(true_conf)
print(class_f1_weighted_scores)
print(f1_scores_dict)

0.9021000465968418
{0: 0.0, 1: 0.8181069245111218, 2: 0.0, 3: 0.0, 4: 0.8038517898262508, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 0.0, 15: 0.0, 16: 0.0, 17: 0.946137760011844, 18: 0.8207965469884245, 19: 0.5152542372881356, 20: 0.6308851224105462, 21: 0.7803129074315516, 22: 0.6215686274509804, 23: 0.5956607495069034, 24: 0.9594594594594594, 25: 0.5911330049261084, 26: 0.6997333333333333, 27: 0.6473665852807814, 28: 0.7735191637630662, 29: 0.6019300361881785, 30: 0.5189620758483035, 31: 0.93922178154143, 32: 0.8230683090705487, 33: 0.0}


In [34]:
true_cnt

array([[     0],
       [ 11366],
       [     0],
       [     0],
       [  2308],
       [     0],
       [     0],
       [     0],
       [     0],
       [     0],
       [     0],
       [     0],
       [     0],
       [     0],
       [     0],
       [     0],
       [     0],
       [107069],
       [ 13106],
       [   641],
       [   494],
       [  1552],
       [   669],
       [   321],
       [  4419],
       [   385],
       [  1883],
       [  1441],
       [  1239],
       [   942],
       [  1500],
       [  4741],
       [  1924],
       [     0]])

In [26]:
#xfer_complete_frompred_unet_mj
tmp=stitch_count('../dataset/instruction-complete',['../dataset/val_real_mj.txt'])
true_cnt=ordered_dict_to_list(tmp,34)
conf=np.array([
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.733,0.000,0.000,0.008,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.186,0.059,0.000,0.000,0.002,0.000,0.000,0.002,0.000,0.003,0.000,0.000,0.001,0.000,0.005,0.001,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.035,0.000,0.000,0.693,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.189,0.028,0.000,0.000,0.027,0.000,0.008,0.006,0.010,0.001,0.000,0.000,0.000,0.000,0.002,0.001,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.016,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.944,0.017,0.000,0.001,0.002,0.000,0.000,0.002,0.000,0.003,0.003,0.002,0.001,0.001,0.003,0.001,0.000],
[0.000,0.047,0.000,0.000,0.006,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.202,0.728,0.000,0.000,0.002,0.000,0.000,0.004,0.001,0.005,0.000,0.000,0.001,0.000,0.002,0.001,0.000],
[0.000,0.019,0.000,0.000,0.009,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.575,0.013,0.180,0.195,0.001,0.009,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.021,0.000,0.000,0.016,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.344,0.004,0.009,0.593,0.001,0.012,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.020,0.000,0.000,0.078,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.401,0.075,0.000,0.000,0.409,0.000,0.002,0.003,0.004,0.001,0.000,0.000,0.000,0.000,0.006,0.000,0.000],
[0.000,0.017,0.000,0.000,0.007,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.523,0.013,0.000,0.100,0.001,0.334,0.002,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.030,0.000,0.000,0.205,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.291,0.030,0.000,0.000,0.022,0.000,0.399,0.006,0.017,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000],
[0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.054,0.011,0.000,0.000,0.003,0.000,0.000,0.927,0.000,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000],
[0.000,0.037,0.000,0.000,0.132,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.390,0.046,0.000,0.001,0.024,0.001,0.006,0.006,0.357,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000],
[0.000,0.024,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.506,0.011,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.376,0.000,0.000,0.036,0.000,0.007,0.039,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.496,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.411,0.061,0.000,0.029,0.002,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.666,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.149,0.173,0.000,0.012,0.000,0.000,0.000],
[0.000,0.014,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.538,0.011,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.169,0.000,0.000,0.223,0.000,0.001,0.043,0.000],
[0.000,0.006,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.715,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.145,0.029,0.000,0.103,0.000,0.000,0.000],
[0.000,0.024,0.000,0.000,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.058,0.005,0.000,0.000,0.004,0.000,0.000,0.002,0.001,0.001,0.000,0.000,0.000,0.000,0.902,0.000,0.000],
[0.000,0.029,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.480,0.015,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.099,0.000,0.000,0.031,0.000,0.001,0.345,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000]])

true_conf=conf*true_cnt
class_f1_weighted_scores, f1_scores_dict = calculate_f1(true_conf)
print(class_f1_weighted_scores)
print(f1_scores_dict)

0.8424529686985092
{0: 0.0, 1: 0.7426457479051524, 2: 0.0, 3: 0.0, 4: 0.6833333333333333, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 0.0, 15: 0.0, 16: 0.0, 17: 0.9167868513428937, 18: 0.7487541691190895, 19: 0.30223390275952694, 20: 0.5400921658986175, 21: 0.49609374999999994, 22: 0.49336283185840707, 23: 0.5412262156448202, 24: 0.9276412637300419, 25: 0.4773519163763066, 26: 0.4200533966182142, 27: 0.42960812772133533, 28: 0.23790994997220677, 29: 0.29787234042553196, 30: 0.16960352422907488, 31: 0.9023952727656431, 32: 0.4679351656095842, 33: 0.0}


In [36]:
#xfer_complete_fromtrue_residual_sj
tmp=stitch_count('../dataset/instruction-complete',['../dataset/val_real_sj.txt'])
true_cnt=ordered_dict_to_list(tmp,34)
conf=np.array([
[0.999,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.003,0.994,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.005,0.004,0.986,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.005,0.967,0.026,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.011,0.975,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.013,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.996,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.042,0.009,0.000,0.932,0.002,0.000,0.000,0.000,0.000,0.005,0.000,0.000,0.005,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.001,0.001,0.995,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.007,0.000,0.000,0.989,0.001,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.005,0.995,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.010,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.987,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.043,0.052,0.000,0.375,0.000,0.000,0.000,0.000,0.000,0.529,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.016,0.063,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.011,0.000,0.909,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.204,0.059,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.738,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.996,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.995,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.049,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.008,0.000,0.000,0.000,0.000,0.000,0.000,0.943,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000]])

true_conf=conf*true_cnt
class_f1_weighted_scores, f1_scores_dict = calculate_f1(true_conf)
print(class_f1_weighted_scores)
print(f1_scores_dict)

0.9979040182868347
{0: 0.9995465855681505, 1: 0.9967014019041907, 2: 0.9899280575539567, 3: 0.9713998996487708, 4: 0.970753655793026, 5: 0.9959758551307847, 6: 0.9483870967741935, 7: 0.9944410129709697, 8: 0.9884259259259259, 9: 0.995945945945946, 10: 1.0, 11: 0.9885990941746057, 12: 0.6, 13: 0.7536231884057972, 14: 0.7199999999999999, 15: 0.996278270827369, 16: 0.997370456303171, 17: 0.0, 18: 0.0, 19: 0.0, 20: 0.0, 21: 0.0, 22: 0.9818181818181818, 23: 0.0, 24: 0.0, 25: 0.0, 26: 0.0, 27: 0.0, 28: 0.0, 29: 0.0, 30: 0.0, 31: 0.0, 32: 0.0, 33: 0.0}


In [28]:
#xfer_complete_fromtrue_unet_sj
tmp=stitch_count('../dataset/instruction-complete',['../dataset/val_real_sj.txt'])
true_cnt=ordered_dict_to_list(tmp,34)
conf=np.array([
[0.998,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.005,0.993,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.006,0.003,0.989,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.936,0.055,0.000,0.005,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.018,0.956,0.000,0.014,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.010,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.998,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.083,0.183,0.000,0.724,0.000,0.000,0.000,0.000,0.000,0.009,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.985,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.013,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.016,0.000,0.000,0.981,0.001,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.001,0.994,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.003,0.996,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.037,0.001,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.955,0.000,0.000,0.000,0.000,0.006,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.107,0.297,0.000,0.109,0.000,0.000,0.000,0.000,0.000,0.487,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.027,0.246,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.004,0.000,0.721,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.752,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.022,0.000,0.226,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.009,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.989,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.029,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.011,0.000,0.000,0.000,0.000,0.960,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.131,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.010,0.000,0.000,0.013,0.000,0.000,0.000,0.000,0.000,0.846,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000]])

true_conf=conf*true_cnt
class_f1_weighted_scores, f1_scores_dict = calculate_f1(true_conf)
print(class_f1_weighted_scores)
print(f1_scores_dict)

0.9931333407242617
{0: 0.9974316629016133, 1: 0.9953906689151208, 2: 0.9928057553956835, 3: 0.9398194583751254, 4: 0.9317180616740088, 5: 0.9942370333249813, 6: 0.7889273356401384, 7: 0.9821318545902651, 8: 0.9854735618826262, 9: 0.9939312204989885, 10: 0.9957924263674615, 11: 0.9576363920587775, 12: 0.39999999999999997, 13: 0.5454545454545455, 14: 0.2962962962962963, 15: 0.9902578796561605, 16: 0.9625502940266171, 17: 0.0, 18: 0.0, 19: 0.0, 20: 0.0, 21: 0.0, 22: 0.7142857142857144, 23: 0.0, 24: 0.0, 25: 0.0, 26: 0.0, 27: 0.0, 28: 0.0, 29: 0.0, 30: 0.0, 31: 0.0, 32: 0.0, 33: 0.0}


In [29]:
#xfer_complete_fromtrue_residual_mj
tmp=stitch_count('../dataset/instruction-complete',['../dataset/val_real_mj.txt'])
true_cnt=ordered_dict_to_list(tmp,34)
conf=np.array([
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.934,0.000,0.000,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.016,0.041,0.002,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.005,0.000,0.000,0.958,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.006,0.019,0.008,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.989,0.003,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.023,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.051,0.923,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.001,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.003,0.001,0.955,0.038,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.050,0.936,0.001,0.010,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.008,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.090,0.000,0.000,0.003,0.883,0.010,0.001,0.005,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.024,0.000,0.000,0.016,0.002,0.957,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.004,0.000,0.000,0.012,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.044,0.000,0.000,0.002,0.011,0.001,0.925,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.999,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.004,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.038,0.004,0.000,0.007,0.005,0.012,0.000,0.000,0.915,0.000,0.000,0.000,0.000,0.012,0.000,0.000,0.000],
[0.000,0.006,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.019,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.795,0.000,0.000,0.112,0.000,0.007,0.062,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.842,0.110,0.000,0.048,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.416,0.570,0.000,0.011,0.002,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.008,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.365,0.000,0.000,0.560,0.000,0.000,0.067,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.415,0.037,0.000,0.543,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.001,0.000,0.998,0.000,0.000],
[0.000,0.020,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.026,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.285,0.001,0.001,0.086,0.000,0.000,0.581,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000]])

true_conf=conf*true_cnt
class_f1_weighted_scores, f1_scores_dict = calculate_f1(true_conf)
print(class_f1_weighted_scores)
print(f1_scores_dict)

0.9596017620975608
{0: 0.0, 1: 0.9282966072053166, 2: 0.0, 3: 0.0, 4: 0.9600521059487626, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 0.0, 15: 0.0, 16: 0.0, 17: 0.9896216408181194, 18: 0.9292518051928099, 19: 0.9272727272727272, 20: 0.9157581764122894, 21: 0.8919270833333333, 22: 0.95952023988006, 23: 0.9580645161290322, 24: 0.9990948178320888, 25: 0.9500674763832659, 26: 0.6826265389876881, 27: 0.6395992617980489, 28: 0.6537037037037038, 29: 0.5701943844492441, 30: 0.678601165695254, 31: 0.9973653704289177, 32: 0.6941943495808754, 33: 0.0}


In [30]:
#xfer_complete_fromtrue_unet_mj
tmp=stitch_count('../dataset/instruction-complete',['../dataset/val_real_mj.txt'])
true_cnt=ordered_dict_to_list(tmp,34)
conf=np.array([
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.927,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.029,0.037,0.002,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.010,0.000,0.000,0.942,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.011,0.014,0.013,0.005,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.988,0.006,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.036,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.046,0.916,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.012,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.019,0.001,0.940,0.028,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.002,0.061,0.918,0.010,0.005,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.033,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.138,0.004,0.000,0.003,0.800,0.001,0.003,0.011,0.002,0.000,0.000,0.005,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.039,0.001,0.000,0.013,0.005,0.937,0.001,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.001,0.000,0.000,0.034,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.029,0.014,0.000,0.014,0.021,0.011,0.858,0.000,0.018,0.001,0.000,0.000,0.001,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.998,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.003,0.000,0.000,0.011,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.061,0.007,0.000,0.001,0.004,0.036,0.001,0.000,0.876,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.130,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.723,0.002,0.000,0.080,0.000,0.003,0.059,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.817,0.143,0.000,0.039,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.516,0.472,0.000,0.009,0.001,0.000,0.000],
[0.000,0.025,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.135,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.256,0.000,0.000,0.551,0.000,0.000,0.032,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.455,0.068,0.000,0.472,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.005,0.000,0.005,0.004,0.000,0.983,0.000,0.000],
[0.000,0.013,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.081,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.121,0.001,0.000,0.030,0.000,0.000,0.754,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000]])

true_conf=conf*true_cnt
class_f1_weighted_scores, f1_scores_dict = calculate_f1(true_conf)
print(class_f1_weighted_scores)
print(f1_scores_dict)

0.9543890705834842
{0: 0.0, 1: 0.9210595331759769, 2: 0.0, 3: 0.0, 4: 0.9460400348128808, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 0.0, 15: 0.0, 16: 0.0, 17: 0.9857381807677434, 18: 0.9156433529097705, 19: 0.90744920993228, 20: 0.9023904382470119, 21: 0.84375, 22: 0.9521640091116174, 23: 0.9151414309484194, 24: 0.9961599277162864, 25: 0.9220246238030096, 26: 0.707012987012987, 27: 0.5970073548059853, 28: 0.5414160111059695, 29: 0.6149289099526066, 30: 0.6229652441706997, 31: 0.9911730298840796, 32: 0.8251350582883139, 33: 0.0}
